In [ ]:
from google.colab import files
uploaded = files.upload()


Saving 202502 Brands Reports v5.xlsx to 202502 Brands Reports v5.xlsx


In [ ]:
import pandas as pd

# Load the entire workbook (if uploaded file is like 'report.xlsx')
xlsx = pd.ExcelFile("/content/202502 Brands Reports v5.xlsx")

# Check available sheet names
print(xlsx.sheet_names)

['ASIN-Brand', 'Business Report', 'Transaction Report', 'Return Report', 'Selling Econ', 'Storage', 'LT Storage', 'Unit Financial', 'Shipping + Label']


In [ ]:
asin_brand_df = pd.read_excel(xlsx, sheet_name="ASIN-Brand")
business_df = pd.read_excel(xlsx, sheet_name="Business Report")
final_business_df = pd.read_csv("BusinessReport-4-24-25.csv")
transaction_df = pd.read_excel(xlsx, sheet_name="Transaction Report", skiprows=7)
return_df = pd.read_excel(xlsx, sheet_name="Return Report")
selling_econ_df = pd.read_excel(xlsx, sheet_name="Selling Econ")
storage_df = pd.read_excel(xlsx, sheet_name="Storage")
lt_storage_df = pd.read_excel(xlsx, sheet_name="LT Storage")
uf = pd.ExcelFile("/content/202502 UF.xlsx")
unit_financial_df = pd.read_excel(xlsx, sheet_name="Unit Financial", skiprows=3)
# unit_financial_df = pd.read_excel(xlsx, sheet_name="Unit Financial", skiprows=3)
shipping_label_df = pd.read_excel(xlsx, sheet_name="Shipping + Label")

In [ ]:
print(transaction_df.columns.tolist())


['date/time', 'settlement id', 'type', 'order id', 'sku', 'description', 'quantity', 'marketplace', 'account type', 'fulfillment', 'order city', 'order state', 'order postal', 'tax collection model', 'product sales', 'product sales tax', 'shipping credits', 'shipping credits tax', 'gift wrap credits', 'giftwrap credits tax', 'Regulatory Fee', 'Tax On Regulatory Fee', 'promotional rebates', 'promotional rebates tax', 'marketplace withheld tax', 'selling fees', 'fba fees', 'other transaction fees', 'other', 'total']


In [ ]:
# Step 1: Filter 'Order' rows
orders_df = transaction_df[transaction_df['type'] == 'Order'].copy()

In [ ]:
# Step 2: Clean monetary columns
to_clean_cols = ['product sales', 'selling fees', 'fba fees', 'other transaction fees', 'quantity']
for col in to_clean_cols:
    orders_df[col] = pd.to_numeric(orders_df[col].replace('[\$,]', '', regex=True), errors='coerce')

In [ ]:
# Step 2: Clean monetary columns
to_clean_cols = ['product sales', 'selling fees', 'fba fees', 'other transaction fees', 'quantity']
for col in to_clean_cols:
    orders_df[col] = pd.to_numeric(
        orders_df[col].replace('[\$,]', '', regex=True),
        errors='coerce'
    )

# Step 3: Prepare SKU → ASIN mapping
sku_asin_map = final_business_df[['SKU', '(Child) ASIN']].dropna()

# Step 4: Detect SKUs mapped to multiple ASINs
conflict_skus = sku_asin_map.groupby('SKU')['(Child) ASIN'].nunique()
conflict_skus = conflict_skus[conflict_skus > 1].index.tolist()

# Step 5: Warn about conflicts
if conflict_skus:
    print(f"⚠️ Warning: {len(conflict_skus)} SKUs map to multiple ASINs. Proceeding by keeping the first ASIN for each SKU.")

# Step 6: Resolve conflicts by keeping the first ASIN
clean_sku_asin_map = sku_asin_map.drop_duplicates(subset='SKU', keep='first')

# Step 7: Merge orders with ASIN mapping
orders_with_asin = clean_sku_asin_map.merge(
    orders_df,
    how='left',
    left_on='SKU',
    right_on='sku'
)

# Step 8: Aggregate numeric columns by ASIN
agg_df = orders_with_asin.groupby('(Child) ASIN')[to_clean_cols].sum().reset_index()


⚠️ Warning: 1 SKUs map to multiple ASINs. Proceeding by keeping the first ASIN for each SKU.


In [ ]:
agg_df

,(Child) ASIN,product sales,selling fees,fba fees,other transaction fees,quantity
0,B00024JRC6,0.0,0.0,0.0,0.0,0.0
1,B000CLYZZ8,0.0,0.0,0.0,0.0,0.0
2,B000HHJGEM,0.0,0.0,0.0,0.0,0.0
3,B000HHLJ24,0.0,0.0,0.0,0.0,0.0
4,B000HHLRGW,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
634,B0DYFX5KRT,0.0,0.0,0.0,0.0,0.0
635,B0DYFX6SGF,0.0,0.0,0.0,0.0,0.0
636,B0F22K57MY,0.0,0.0,0.0,0.0,0.0
637,B0F22KW4C1,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Step 9: Find total Subscription + Premium Services Fees correctly
# Filter Subscription Fee and Premium Services Fee separately
subscription_fee_df = transaction_df[transaction_df['description'] == 'Subscription'].copy()
premium_service_fee_df = transaction_df[transaction_df['description'] == 'Premium Services Fee'].copy()

# Clean the relevant columns
subscription_fee_df['other transaction fees'] = pd.to_numeric(
    subscription_fee_df['other transaction fees'].replace('[\$,]', '', regex=True),
    errors='coerce'
).fillna(0)

premium_service_fee_df['selling fees'] = pd.to_numeric(
    premium_service_fee_df['selling fees'].replace('[\$,]', '', regex=True),
    errors='coerce'
).fillna(0)

# Sum separately
total_subscription_fee = subscription_fee_df['other'].sum()
total_premium_service_fee = premium_service_fee_df['selling fees'].sum()

# Add together
total_subscription_premium_fee = total_subscription_fee + total_premium_service_fee

print(f"✅ Total Subscription Fee (from Other Fees) = {total_subscription_fee}")
print(f"✅ Total Premium Service Fee (from Selling Fees) = {total_premium_service_fee}")
print(f"✅ Grand Total = {total_subscription_premium_fee}")

# Step 10: Find total product sales (across all ASINs)
total_product_sales = agg_df['product sales'].sum()
print(total_product_sales)
# Step 11: Calculate Allocatable Fees per ASIN and round to 2 decimals
agg_df['Allocatable Fees'] = (agg_df['product sales'] / total_product_sales * total_subscription_premium_fee).round(2)


✅ Total Subscription Fee (from Other Fees) = -18.58
✅ Total Premium Service Fee (from Selling Fees) = -5000.0
✅ Grand Total = -5018.58
1180483.2


In [ ]:
# Debug Check: is your ASIN present?
check_asin = 'B01E46PJCE'
if (agg_df['(Child) ASIN'] == check_asin).any():
    print(f"✅ ASIN {check_asin} exists in agg_df.")
else:
    print(f"❌ ASIN {check_asin} NOT found in agg_df.")

✅ ASIN B01E46PJCE exists in agg_df.


In [ ]:
# Optional: Validate row increase (debugging)
print("Orders before merge:", len(orders_df))
print("Orders after merge :", len(orders_with_asin))

Orders before merge: 20423
Orders after merge : 20940


In [ ]:
# Step 7: Compute Gross Profit
agg_df['Gross Profit'] = (
    agg_df['product sales']
    + agg_df['selling fees']
    + agg_df['fba fees']
    + agg_df['other transaction fees']
)

In [ ]:
agg_df.head()

,(Child) ASIN,product sales,selling fees,fba fees,other transaction fees,quantity,Allocatable Fees,Gross Profit
0,B00024JRC6,0.0,0.0,0.0,0.0,0.0,-0.0,0.0
1,B000CLYZZ8,0.0,0.0,0.0,0.0,0.0,-0.0,0.0
2,B000HHJGEM,0.0,0.0,0.0,0.0,0.0,-0.0,0.0
3,B000HHLJ24,0.0,0.0,0.0,0.0,0.0,-0.0,0.0
4,B000HHLRGW,0.0,0.0,0.0,0.0,0.0,-0.0,0.0


In [ ]:
# Filter for the specific ASIN
asin_to_check = 'B07HWTT4WY'
asin_row = agg_df[agg_df['(Child) ASIN'] == asin_to_check]
asin_row.head()

,(Child) ASIN,product sales,selling fees,fba fees,other transaction fees,quantity,Allocatable Fees,Gross Profit
242,B07HWTT4WY,16433.52,-2465.1,-1254.41,0.0,166.0,-69.86,12714.01


In [ ]:
unit_financial_df

,Brand Partner,UPC,SKU,ASIN,Product Name,Category,Trademark Brands,REVIEWED SIGN OFF,Fulfilled By,Unnamed: 9,...,Sales Price,Longest Side,Median Side,Shortest Side,Length and Girth,Unnamed: 51,Item Package Weight,Unnamed: 53,Actual Product Size Tier,Currency
0,Animed,694244-97360-4,RB-ANI-097360,B07PH9Z1VG,AniFlex Complete HEMP - 16oz,Pet Products,Animed,NaN,FBA,NaN,...,NaN,5.50,4.00,4.00,16.00,inches,24.00,oz,Large Standard,USD
1,Animed,694244-97361-1,RB-ANI-097361,B07QQKKVXQ,AniFlex Complete HEMP - 2.5 lb,Pet Products,Animed,NaN,FBA,NaN,...,NaN,6.00,5.00,5.00,20.00,inches,48.00,oz,Large Standard,USD
2,Animed,694244-97362-8,RB-ANI-097362,0,AniFlex Complete HEMP - 5 lb,Pet Products,Animed,NaN,FBA,NaN,...,x,7.42,6.97,6.97,27.88,inches,88.00,oz,Large Standard,USD
3,Animed,694244-97011-5,RB-ANI-097011,B07PYF9RZ6,Arthaway HEMP - 16oz,Pet Products,Animed,NaN,FBA,NaN,...,x,5.50,4.00,4.00,16.00,inches,24.00,oz,Large Standard,USD
4,Animed,694244-97415-1,RB-ANI-097415,0,Glucosamine 5000 HEMP - 16oz,Pet Products,Animed,NaN,FBA,NaN,...,x,5.50,4.00,4.00,16.00,inches,24.00,oz,Large Standard,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,CRICUT,2010171,TW-CRI-2010171,B0BTZZYF4P,EDIO SKINTONE SAMPLER 12X12 (10),Everything Else,CRICUT,NaN,FBA,NaN,...,NaN,13.80,1.81,1.81,7.24,inches,10.08,oz,Large Standard,USD
1871,CRICUT,2010196,TW-CRI-2010196,B0BV11HL4J,EVDY IO CORAL 12X24 (1),Everything Else,CRICUT,NaN,FBA,NaN,...,NaN,12.00,1.75,1.75,7.00,inches,3.68,oz,Large Standard,USD
1872,CRICUT,2010037,TW-CRI-2010037,B0BV18GVCK,LINERLESS TRANSFER TAPE 13INx30FT,Everything Else,CRICUT,NaN,FBA,NaN,...,NaN,13.00,2.00,2.00,8.00,inches,12.80,oz,Large Standard,USD
1873,CRICUT,2008875,TW-CRI-2008875,B0CCJVLWR3,PERM MARKERS 2.5 BLACK (3),Everything Else,CRICUT,NaN,FBA,NaN,...,NaN,6.30,2.36,0.83,6.38,inches,2.56,oz,Large Standard,USD


In [ ]:
# Step 6: Clean and map unit cost
unit_financial_df['Current Wholesale Price'] = pd.to_numeric(
    unit_financial_df['Current Wholesale Price'].replace('[\$,]', '', regex=True), errors='coerce'
)
unit_cost_map = unit_financial_df[['ASIN', 'Current Wholesale Price']].drop_duplicates(subset='ASIN', keep='first')

# Merge unit cost into agg_df
agg_df = agg_df.merge(unit_cost_map, how='left', left_on='(Child) ASIN', right_on='ASIN')

# Step 7: Compute Product Cost as negative
agg_df['Product Cost'] = -agg_df['Current Wholesale Price'] * agg_df['quantity']

# Step 8: Ensure all ASINs are present
all_asins = final_business_df[['(Child) ASIN']].drop_duplicates()
agg_df = all_asins.merge(agg_df, on='(Child) ASIN', how='left').fillna(0)

# Step 9: Drop redundant columns
agg_df.drop(columns=['ASIN'], inplace=True)

In [ ]:
agg_df.head(10)

,(Child) ASIN,product sales,selling fees,fba fees,other transaction fees,quantity,Allocatable Fees,Gross Profit,Current Wholesale Price,Product Cost
0,B01E46PJCE,136654.77,-19274.58,-9299.90,0.0,2153.0,-580.96,108080.29,42.24,-90942.72
1,B01E46WJIG,87232.27,-12359.24,-6484.74,0.0,1595.0,-370.85,68388.29,36.39,-58042.05
2,B01E44ZX9U,93948.52,-13218.54,-6090.84,0.0,1293.0,-399.40,74639.14,48.39,-62568.27
3,B08MFNJSPG,667.23,-100.11,-112.05,0.0,27.0,-2.84,455.07,12.71,-343.17
4,B0BHC99KGY,112884.05,-16132.85,-7549.79,0.0,1745.0,-479.90,89201.41,43.72,-76291.40
5,B0BHBNLNM1,82424.67,-11806.66,-5679.12,0.0,1479.0,-350.41,64938.89,37.66,-55699.14
6,B0BHBW54WV,88525.60,-12667.29,-5634.10,0.0,1195.0,-376.35,70224.21,50.07,-59833.65
7,B0BL56DXNX,0.00,0.00,0.00,0.0,0.0,-0.00,0.00,3.30,-0.00
8,B07W7646SJ,1841.73,-275.83,-339.39,0.0,77.0,-7.83,1226.51,12.71,-978.67
9,B004JR1YJQ,21815.17,-3209.13,-2385.53,0.0,485.0,-92.74,16220.51,22.93,-11121.05


In [ ]:
agg_df.drop(columns=[
    'Featured Offer (Buy Box) Percentage',
    'Sessions - Total',
    'Unit Session Percentage'
], errors='ignore', inplace=True)

# Ensure unique ASINs for the metrics before merging
business_metrics = final_business_df[['(Child) ASIN', 'Featured Offer (Buy Box) Percentage', 'Sessions - Total', 'Unit Session Percentage']]
business_metrics = business_metrics.drop_duplicates(subset='(Child) ASIN', keep='first')  # or 'first'

# Now merge safely
agg_df = agg_df.merge(
    business_metrics,
    on='(Child) ASIN',
    how='left'
)

In [ ]:
# Check if 'asin' and 'fba fees' are columns in your DataFrame
if '(Child) ASIN' in agg_df.columns and 'product sales' in agg_df.columns:
    # Filter the rows where asin == 'B01E46PJCE'
    asin_fba_fees = agg_df[agg_df['(Child) ASIN'] == 'B012BSPHS8'][['(Child) ASIN', 'product sales']]

    # Display the result
    print(asin_fba_fees)
else:
    print("Make sure 'asin' and 'fba fees' columns exist in agg_df.")

   (Child) ASIN  product sales
75   B012BSPHS8         462.13


In [ ]:
agg_df.head(10)

,(Child) ASIN,product sales,selling fees,fba fees,other transaction fees,quantity,Allocatable Fees,Gross Profit,Current Wholesale Price,Product Cost,Featured Offer (Buy Box) Percentage,Sessions - Total,Unit Session Percentage
0,B01E46PJCE,136654.77,-19274.58,-9299.90,0.0,2153.0,-580.96,108080.29,42.24,-90942.72,92.84%,"59,330",61.97%
1,B01E46WJIG,87232.27,-12359.24,-6484.74,0.0,1595.0,-370.85,68388.29,36.39,-58042.05,94.20%,"55,537",51.22%
2,B01E44ZX9U,93948.52,-13218.54,-6090.84,0.0,1293.0,-399.40,74639.14,48.39,-62568.27,92.75%,"43,542",55.70%
3,B08MFNJSPG,667.23,-100.11,-112.05,0.0,27.0,-2.84,455.07,12.71,-343.17,96.35%,"195,716",10.03%
4,B0BHC99KGY,112884.05,-16132.85,-7549.79,0.0,1745.0,-479.90,89201.41,43.72,-76291.40,97.37%,"54,908",33.75%
5,B0BHBNLNM1,82424.67,-11806.66,-5679.12,0.0,1479.0,-350.41,64938.89,37.66,-55699.14,96.76%,"42,237",36.68%
6,B0BHBW54WV,88525.60,-12667.29,-5634.10,0.0,1195.0,-376.35,70224.21,50.07,-59833.65,94.08%,"48,746",26.41%
7,B0BL56DXNX,0.00,0.00,0.00,0.0,0.0,-0.00,0.00,3.30,-0.00,93.54%,"110,507",11.87%
8,B07W7646SJ,1841.73,-275.83,-339.39,0.0,77.0,-7.83,1226.51,12.71,-978.67,99.72%,"129,430",9.65%
9,B004JR1YJQ,21815.17,-3209.13,-2385.53,0.0,485.0,-92.74,16220.51,22.93,-11121.05,82.40%,"30,498",35.83%


In [ ]:
filtered_df_test = unit_financial_df[unit_financial_df['ASIN'] == 'B088D3295H']
filtered_price = filtered_df_test['Current Wholesale Price']
print(filtered_price)

576    3.24
Name: Current Wholesale Price, dtype: float64


In [ ]:
filtered_df_test2 = agg_df[agg_df['(Child) ASIN'] == 'B088D3295H']
filtered_price2 = filtered_df_test['Current Wholesale Price']
print(filtered_price2)

576    3.24
Name: Current Wholesale Price, dtype: float64


In [ ]:
# Define output file path
output_path = 'aggregated_product_cost_report.xlsx'

# Export agg_df to Excel
agg_df.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: aggregated_product_cost_report.xlsx


In [ ]:
# Step 2: Convert 'amount-charged' to numeric (if needed)
lt_storage_df['amount-charged'] = pd.to_numeric(lt_storage_df['amount-charged'], errors='coerce')

# Step 3: Group by 'asin' and sum the 'amount-charged'
lt_storage_sum = lt_storage_df.groupby('asin')['amount-charged'].sum().reset_index()

# Step 4: Rename the column
lt_storage_sum.rename(columns={'amount-charged': 'long-term storage amount'}, inplace=True)

# Optional: Round to whole dollars
#lt_storage_sum['long-term storage amount'] = lt_storage_sum['long-term storage amount'].round(0).astype(int)

In [ ]:
lt_storage_sum.head()

,asin,long-term storage amount
0,B00B2FCFD2,0.02
1,B00C8RJ3BA,152.69
2,B00DCV38HQ,1.58
3,B01A1F9U86,0.25
4,B01HVFUSXA,0.06


In [ ]:
# Define output path
output_path = 'long_term_storage_summary.xlsx'

# Export to Excel
lt_storage_sum.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: long_term_storage_summary.xlsx


In [ ]:
# Step 1: Clean + round storage fee
storage_df['estimated_monthly_storage_fee'] = pd.to_numeric(
    storage_df['estimated_monthly_storage_fee'], errors='coerce'
).round(8)

# Step 1.5: Clean ASIN formatting
storage_df['asin'] = storage_df['asin'].astype(str).str.strip().str.upper()

# ✅ Step 1.6: Neglect small rows (< 0.001)
storage_df = storage_df[storage_df['estimated_monthly_storage_fee'].abs() >= 0.001]

# ✅ Step 1.7: Clean and sum average quantity on hand per ASIN
storage_df['average_quantity_on_hand'] = pd.to_numeric(
    storage_df['average_quantity_on_hand'], errors='coerce'
).fillna(0)

avg_qty_df = storage_df.groupby('asin', as_index=False)['average_quantity_on_hand'].sum()
avg_qty_df.rename(columns={'average_quantity_on_hand': 'average_quantity_on_hand'}, inplace=True)

# Step 2: Sum estimated fee per ASIN
storage_summary = storage_df.groupby('asin', as_index=False)['estimated_monthly_storage_fee'].sum()

# ✅ Merge with total_average_quantity_on_hand
storage_summary = storage_summary.merge(avg_qty_df, on='asin', how='left')

# Step 3: Total estimated fee
total_estimated_fee = round(storage_summary['estimated_monthly_storage_fee'].sum(), 5)

# Step 4: Check specific ASIN
asin_storage = storage_summary[storage_summary['asin'] == 'B085SLZPC7']
print(asin_storage['estimated_monthly_storage_fee'])
print(total_estimated_fee)
print(660.8882 / total_estimated_fee)

95    660.8882
Name: estimated_monthly_storage_fee, dtype: float64
7160.3142
0.0922987709114776


/tmp/ipython-input-27-2313399053.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storage_df['average_quantity_on_hand'] = pd.to_numeric(


In [ ]:
# Step 4: Compute allocation percentage
storage_summary['allocation %'] = (storage_summary['estimated_monthly_storage_fee'] / total_estimated_fee)

# Step 5: Clean 'other' column in transaction_df
transaction_df['other'] = pd.to_numeric(transaction_df['other'].replace('[\$,]', '', regex=True), errors='coerce')

# Step 6: Find actual total FBA storage fee from transaction description
actual_amount = transaction_df[
    transaction_df['description'].str.contains('FBA storage fee', case=False, na=False)
]['other'].sum()
print(actual_amount)

# Step 7: Allocate the actual amount using the allocation percentage
storage_summary['fee $'] = (storage_summary['allocation %'] * actual_amount).round(2)

# Optional: Format columns
storage_summary['allocation %'] = (storage_summary['allocation %'] * 100).round(2).astype(str) + '%'

-6980.75


In [ ]:
print(-6980.75*0.00575289)

-40.1594868675


In [ ]:
storage_summary.head()

,asin,estimated_monthly_storage_fee,average_quantity_on_hand,allocation %,fee $
0,B00024JRC6,4.7353,19.35,0.07%,-4.62
1,B001ODZ5BQ,0.0062,0.04,0.0%,-0.01
2,B002NU6FJK,1.5900,5.00,0.02%,-1.55
3,B004JR1YJQ,5.8741,179.03,0.08%,-5.73
4,B0056EAKSM,2.2764,218.51,0.03%,-2.22


In [ ]:
# Filter for ASIN 'B01E46PJCE' and show relevant columns
asin_storage = storage_summary[storage_summary['asin'] == 'B085SLZPC7'][[
    'asin',
    'estimated_monthly_storage_fee',
    'allocation %',
    'fee $'
]]

# Display the result
print(asin_storage)


          asin  estimated_monthly_storage_fee allocation %   fee $
95  B085SLZPC7                       660.8882        9.23% -644.31


In [ ]:
import pandas as pd

# Step 1: Read the main inventory file
file_path = 'Brands FBA Inventory.xlsx'
df_inventory = pd.read_excel(file_path, sheet_name='934653020206')

# Step 3: Group the inventory data
grouped_inventory = df_inventory.groupby('asin').agg({
    'product-name': 'first',  # Pick the first name (usually the same per ASIN)
    'available': 'sum',
    'inbound-quantity': 'sum',
    'Total Reserved Quantity': 'sum',
    'fba-inventory-level-health-status': lambda x: ', '.join(sorted(x.dropna().astype(str).unique()))
}).reset_index()

# Step 4: Merge with storage_summary
# Assume storage_summary has columns ['asin', 'Sum of estimated_monthly_storage_fee']
# Rename 'Sum of estimated_monthly_storage_fee' during merge
storage_summary_renamed = storage_summary[['asin', 'estimated_monthly_storage_fee']].copy()
storage_summary_renamed.rename(columns={
    'estimated_monthly_storage_fee': 'Estimated monthly storage per unit'
}, inplace=True)

final_df = grouped_inventory.merge(storage_summary_renamed, on='asin', how='left')
final_df = final_df.fillna(0)
final_df


,asin,product-name,available,inbound-quantity,Total Reserved Quantity,fba-inventory-level-health-status,Estimated monthly storage per unit
0,B00024JRC6,"Buddeez Bread Buddy Bread Box – White Lid, Pac...",0,0,16,Out of stock,4.7353
1,B004C0OHZS,Kijaro XXL Dual Lock Portable Camping Chair - ...,1,0,0,Low stock,0.0000
2,B004C0QECC,Kijaro Dual Lock Portable Camping Chairs - Enj...,1,0,0,Low stock,0.0000
3,B004JR1YJQ,Myogenix Myovite Multivitamins for Athletes - ...,0,3007,766,Low stock,5.8741
4,B0056EAKSM,"Virbac Epi-Otic Advanced Ear Cleaner, 8 oz",356,0,32,Low stock,2.2764
...,...,...,...,...,...,...,...
174,B0DVMSZXDT,Virbac URSOLYX Soft Chews for Dogs | Muscle He...,0,132,47,Low stock,0.0000
175,B0DWBXX6WY,Learning Dynamics 53 Decodable Reading Books &...,7,10,6,Low stock,0.0000
176,B0DWCFSX83,Manual de Lecciones en Español de Learning Dyn...,10,0,10,,0.0000
177,B0DYFD1TRN,Virbac URSOLYX Soft Chews for Dogs | Muscle He...,0,12,1,Out of stock,0.0000


In [ ]:
# Step 5: Save the result
final_df.to_excel('Processed_Brands_FBA_Inventory.xlsx', index=False)

print("✅ Done! Output saved to 'Processed_Brands_FBA_Inventory.xlsx'")

✅ Done! Output saved to 'Processed_Brands_FBA_Inventory.xlsx'


In [ ]:
# Define output path
output_path = 'monthly_fba_storage_allocation.xlsx'

# Export to Excel
storage_summary.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: monthly_fba_storage_allocation.xlsx


In [ ]:
# Step 8: Prepare short-term storage data
short_term_df = storage_summary[['asin', 'fee $']].copy()
short_term_df.rename(columns={'fee $': 'short-term storage amount'}, inplace=True)

# Step 9: Prepare long-term storage data (from lt_storage_sum)
lt_storage_sum['long-term storage amount'] = lt_storage_sum['long-term storage amount'].round(2)

# Step 10: Merge short-term and long-term storage costs
total_storage_df = pd.merge(
    short_term_df, lt_storage_sum, how='outer', on='asin'
)

# Step 11: Fill any missing values with 0
total_storage_df.fillna(0, inplace=True)

# Step 12: Calculate total storage cost per ASIN
total_storage_df['total storage cost'] = (
    -total_storage_df['short-term storage amount'] + total_storage_df['long-term storage amount']
)
total_storage_df['long-term storage amount'] = total_storage_df['long-term storage amount'].apply(
    lambda x: 0 if x == 0 else -abs(x)
)
total_storage_df['total storage cost'] = total_storage_df['total storage cost'].apply(
    lambda x: 0 if x == 0 else -abs(x)
)
# Step 13: Round total storage cost to 2 decimal places
total_storage_df['short-term storage amount'] = total_storage_df['short-term storage amount'].round(2)
total_storage_df['total storage cost'] = total_storage_df['total storage cost'].round(2)


In [ ]:
total_storage_df.head()

,asin,short-term storage amount,long-term storage amount,total storage cost
0,B00024JRC6,-4.62,0.0,-4.62
1,B001ODZ5BQ,-0.01,0.0,-0.01
2,B002NU6FJK,-1.55,0.0,-1.55
3,B004JR1YJQ,-5.73,0.0,-5.73
4,B0056EAKSM,-2.22,0.0,-2.22


In [ ]:
# Define output path
output_path = 'total_storage_cost_per_asin.xlsx'

# Export to Excel
total_storage_df.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: total_storage_cost_per_asin.xlsx


In [ ]:
# Step 1: Ensure both DataFrames have a common column name for the join
agg_subset = agg_df.rename(columns={"(Child) ASIN": "ASIN"})
total_storage_df = total_storage_df.rename(columns={"asin": "ASIN"})

# Step 2: Perform outer join
merged_df = agg_subset.merge(total_storage_df[['ASIN', 'total storage cost']], on='ASIN', how='left')


In [ ]:
merged_df.head(20)


,ASIN,product sales,selling fees,fba fees,other transaction fees,quantity,Allocatable Fees,Gross Profit,Current Wholesale Price,Product Cost,Featured Offer (Buy Box) Percentage,Sessions - Total,Unit Session Percentage,total storage cost
0,B01E46PJCE,136654.77,-19274.58,-9299.90,0.0,2153.0,-580.96,108080.29,42.24,-90942.72,92.84%,"59,330",61.97%,-40.16
1,B01E46WJIG,87232.27,-12359.24,-6484.74,0.0,1595.0,-370.85,68388.29,36.39,-58042.05,94.20%,"55,537",51.22%,-14.26
2,B01E44ZX9U,93948.52,-13218.54,-6090.84,0.0,1293.0,-399.40,74639.14,48.39,-62568.27,92.75%,"43,542",55.70%,-33.76
3,B08MFNJSPG,667.23,-100.11,-112.05,0.0,27.0,-2.84,455.07,12.71,-343.17,96.35%,"195,716",10.03%,-9.34
4,B0BHC99KGY,112884.05,-16132.85,-7549.79,0.0,1745.0,-479.90,89201.41,43.72,-76291.40,97.37%,"54,908",33.75%,-26.02
5,B0BHBNLNM1,82424.67,-11806.66,-5679.12,0.0,1479.0,-350.41,64938.89,37.66,-55699.14,96.76%,"42,237",36.68%,-10.16
6,B0BHBW54WV,88525.60,-12667.29,-5634.10,0.0,1195.0,-376.35,70224.21,50.07,-59833.65,94.08%,"48,746",26.41%,-18.19
7,B0BL56DXNX,0.00,0.00,0.00,0.0,0.0,-0.00,0.00,3.30,-0.00,93.54%,"110,507",11.87%,-3.72
8,B07W7646SJ,1841.73,-275.83,-339.39,0.0,77.0,-7.83,1226.51,12.71,-978.67,99.72%,"129,430",9.65%,-31.30
9,B004JR1YJQ,21815.17,-3209.13,-2385.53,0.0,485.0,-92.74,16220.51,22.93,-11121.05,82.40%,"30,498",35.83%,-5.73


In [ ]:
# Filter for ASIN 'B01E46PJCE' and show relevant columns
asin_storage = merged_df[merged_df['ASIN'] == 'B0056EAKSM'][[
    'ASIN',
    'total storage cost'
]]

# Display the result
print(asin_storage)


          ASIN  total storage cost
61  B0056EAKSM               -2.22


In [ ]:
# Define output file path
output_path = 'aggregated_product_cost_report2.xlsx'

# Export agg_df to Excel
merged_df.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: aggregated_product_cost_report2.xlsx


In [ ]:
import pandas as pd
import os

# Define the directory containing Excel files
data_dir = "/content/data"

# Define required columns
required_columns = [
    "FBA inbound placement service fee per unit",
    "Inbound Transportation Fee per unit",
    "Label per unit",
    "Bagging per unit",
     "FBA disposal order fee per unit",
    "FBA removal order fee per unit",
    "Refund administration fee per unit",
    "Returns Processing Fee for Non-Apparel and Non-Shoes per unit",
    "Returns processing fee for Apparel and Shoes per unit"
]

# Read all Excel files in the directory and concatenate
all_dataframes = []
for file in os.listdir(data_dir):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(data_dir, file))

        # Ensure required columns exist
        for col in required_columns:
            if col not in df.columns:
                df[col] = 0

        # Convert date columns to datetime
        df["Start date"] = pd.to_datetime(df["Start date"], errors='coerce')
        df["End date"] = pd.to_datetime(df["End date"], errors='coerce')

        all_dataframes.append(df)

# ✅ Add Selling Econ as an additional source
selling_econ_df = pd.read_excel(xlsx, sheet_name="Selling Econ")
for col in required_columns:
    if col not in selling_econ_df.columns:
        selling_econ_df[col] = 0
selling_econ_df["Start date"] = pd.to_datetime(selling_econ_df["Start date"], errors='coerce')
selling_econ_df["End date"] = pd.to_datetime(selling_econ_df["End date"], errors='coerce')
all_dataframes.append(selling_econ_df)

# Combine all dataframes
combined_df = pd.concat(all_dataframes, ignore_index=True)

# Sort by 'End date' descending (most recent first)
combined_df.sort_values(by="End date", ascending=False, inplace=True)

# Reset index after sorting
combined_df.reset_index(drop=True, inplace=True)



In [ ]:
combined_df.head()

,Amazon store,Start date,End date,Parent ASIN,ASIN,FNSKU,MSKU,Currency code,Average sales price,Units sold,...,Returns processing fee for Apparel and Shoes total,Storage utilization surcharge per unit,Storage utilization surcharge quantity,Storage utilization surcharge total,Sponsored Products charge per unit,Sponsored Products charge quantity,Sponsored Products charge total,FBA removal order fee per unit,FBA removal order fee quantity,FBA removal order fee total
0,US,2025-02-01,2025-02-28,B0F22K57MY,B0F22K57MY,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,US,2025-02-01,2025-02-28,B06XBGXLHR,B06XBGXLHR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,US,2025-02-01,2025-02-28,B09FWBGPVS,B075K58RRG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,US,2025-02-01,2025-02-28,B0758JKQRQ,B0758JKQRQ,NaN,NaN,USD,NaN,NaN,...,NaN,0.0,28.00,0.0,NaN,NaN,NaN,0.0,NaN,NaN
4,US,2025-02-01,2025-02-28,B07525YD4C,B07525YD4C,NaN,NaN,USD,NaN,NaN,...,NaN,0.0,0.82,0.0,NaN,NaN,NaN,0.0,NaN,NaN


In [ ]:
recent_values = {}
threshold = 0.0001  # Non-zero threshold

# 1. Sort once globally
sorted_all = combined_df.sort_values(by=["ASIN", "End date"], ascending=[True, False]).copy()

# 2. Loop through required columns
for col in required_columns:
    asin_values = {}

    for asin, asin_df in sorted_all.groupby('ASIN'):
        # ✅ Very important: Sort each ASIN's rows again by End date descending
        asin_df = asin_df.sort_values(by="End date", ascending=False)

        value_series = asin_df[col]

        # Take first non-zero value
        non_zero_values = value_series[value_series > threshold]

        if not non_zero_values.empty:
            asin_values[asin] = non_zero_values.iloc[0]  # most recent non-zero
        else:
            asin_values[asin] = 0

    recent_values[col] = pd.Series(asin_values)

# 3. Combine into final DataFrame
recent_values_df = pd.DataFrame(recent_values)

# ✅ Keep ASIN as normal column
recent_values_df.index.name = 'ASIN'
recent_values_df = recent_values_df.reset_index()

# 4. Fill NaNs with 0
recent_values_df = recent_values_df.fillna(0)

# 5. Add Shipping Fee and Label Fee
recent_values_df["Shipping Fee"] = (
    recent_values_df["FBA inbound placement service fee per unit"] +
    recent_values_df["Inbound Transportation Fee per unit"]
)

recent_values_df["Label Fee"] = (
    recent_values_df["Label per unit"] +
    recent_values_df["Bagging per unit"]
)

# 6. Final display
recent_values_df.head()


,ASIN,FBA inbound placement service fee per unit,Inbound Transportation Fee per unit,Label per unit,Bagging per unit,FBA disposal order fee per unit,FBA removal order fee per unit,Refund administration fee per unit,Returns Processing Fee for Non-Apparel and Non-Shoes per unit,Returns processing fee for Apparel and Shoes per unit,Shipping Fee,Label Fee
0,B00024JRC6,0.0,0.0,0.0,0.0,1.53,0.0,0.38,0.0,0.0,0.0,0.0
1,B000CLYZZ8,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,B000GCI9TG,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,B000HHJGEM,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
4,B000HHJGFQ,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [ ]:
# Define the 4 important columns
important_cols = [
    'FBA inbound placement service fee per unit',
    'Inbound Transportation Fee per unit',
    'Label per unit',
    'Bagging per unit'
]

# Filter all rows where ASIN is 'B004JR1YJQ'
asin_rows = recent_values_df[recent_values_df['ASIN'] == 'B004JR1YJQ']

# Print only the 4 columns
print(asin_rows[important_cols])


    FBA inbound placement service fee per unit  \
63                                        0.27   

    Inbound Transportation Fee per unit  Label per unit  Bagging per unit  
63                                 0.65             0.6               0.0  


In [ ]:
# Step 2: Merge Shipping Fee and Label Fee into merged_df
final_df = merged_df.merge(
    recent_values_df[['ASIN', 'Shipping Fee', 'Label Fee']],
    on='ASIN',
    how='left'
)
# Multiply Shipping Fee and Label Fee by quantity
final_df['Shipping Fee'] = -final_df['Shipping Fee'] * final_df['quantity']
final_df['Label Fee'] = -final_df['Label Fee'] * final_df['quantity']
final_df['Shipping Fee'] = final_df['Shipping Fee'].apply(lambda x: 0 if x == 0 else (-abs(x)))
final_df['Label Fee'] = final_df['Label Fee'].apply(lambda x: 0 if x == 0 else (-abs(x)))
final_df.head(20)

,ASIN,product sales,selling fees,fba fees,other transaction fees,quantity,Allocatable Fees,Gross Profit,Current Wholesale Price,Product Cost,Featured Offer (Buy Box) Percentage,Sessions - Total,Unit Session Percentage,total storage cost,Shipping Fee,Label Fee
0,B01E46PJCE,136654.77,-19274.58,-9299.90,0.0,2153.0,-580.96,108080.29,42.24,-90942.72,92.84%,"59,330",61.97%,-40.16,0.00,0.0
1,B01E46WJIG,87232.27,-12359.24,-6484.74,0.0,1595.0,-370.85,68388.29,36.39,-58042.05,94.20%,"55,537",51.22%,-14.26,0.00,0.0
2,B01E44ZX9U,93948.52,-13218.54,-6090.84,0.0,1293.0,-399.40,74639.14,48.39,-62568.27,92.75%,"43,542",55.70%,-33.76,0.00,0.0
3,B08MFNJSPG,667.23,-100.11,-112.05,0.0,27.0,-2.84,455.07,12.71,-343.17,96.35%,"195,716",10.03%,-9.34,0.00,0.0
4,B0BHC99KGY,112884.05,-16132.85,-7549.79,0.0,1745.0,-479.90,89201.41,43.72,-76291.40,97.37%,"54,908",33.75%,-26.02,0.00,0.0
5,B0BHBNLNM1,82424.67,-11806.66,-5679.12,0.0,1479.0,-350.41,64938.89,37.66,-55699.14,96.76%,"42,237",36.68%,-10.16,0.00,0.0
6,B0BHBW54WV,88525.60,-12667.29,-5634.10,0.0,1195.0,-376.35,70224.21,50.07,-59833.65,94.08%,"48,746",26.41%,-18.19,0.00,0.0
7,B0BL56DXNX,0.00,0.00,0.00,0.0,0.0,-0.00,0.00,3.30,-0.00,93.54%,"110,507",11.87%,-3.72,0.00,0.0
8,B07W7646SJ,1841.73,-275.83,-339.39,0.0,77.0,-7.83,1226.51,12.71,-978.67,99.72%,"129,430",9.65%,-31.30,0.00,0.0
9,B004JR1YJQ,21815.17,-3209.13,-2385.53,0.0,485.0,-92.74,16220.51,22.93,-11121.05,82.40%,"30,498",35.83%,-5.73,-446.20,-291.0


In [ ]:
# Filter the rows where ASIN is 'B004JR1YJQ'
asin_fees = final_df[final_df['ASIN'] == 'B07X7HBL5N'][['ASIN', 'Shipping Fee', 'Label Fee']]

# Display the result
print(asin_fees)

          ASIN  Shipping Fee  Label Fee
69  B07X7HBL5N          -5.1        0.0


In [ ]:
# Define the output Excel path
output_path = 'final_aggregated.xlsx'

# Export to Excel
final_df.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: final_aggregated.xlsx


In [ ]:
# Step 2: Filter refund transactions
refund_df = transaction_df[transaction_df['type'] == 'Refund'].copy()

# Step 3: Clean numeric columns
refund_df['product sales'] = pd.to_numeric(refund_df['product sales'].replace('[\$,]', '', regex=True), errors='coerce')
refund_df['quantity'] = pd.to_numeric(refund_df['quantity'], errors='coerce')

# Ensure business_df has unique SKU-to-ASIN mapping
business_unique = final_business_df[['SKU', '(Child) ASIN']].drop_duplicates(subset='SKU')


# Step 4: Merge to get ASIN
refund_with_asin = refund_df.merge(business_unique, how='left', left_on='sku', right_on='SKU')

# # ✅ Remove duplicates before grouping to prevent double-counting
# refund_with_asin = refund_with_asin.drop_duplicates(
#     subset=["order id", "sku", "type", "product sales", "quantity"]
# )


In [ ]:
suspect = ['B0BHBNLNM1', '']
mask = refund_with_asin['(Child) ASIN'].isin(suspect)
cols = ['order id', 'sku',
        'quantity', 'product sales', '(Child) ASIN']
print(refund_with_asin.loc[mask, cols]
      .sort_values(['order id', 'sku'])
      .head(30))


                order id           sku  quantity  product sales (Child) ASIN
147  111-1785888-6660265  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
207  111-5738090-0270652  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
313  111-9059895-2613016  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
296  112-4482600-2957815  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
241  112-7399576-3331446  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
291  112-8489830-9161855  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
275  113-5886141-6903457  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
276  113-5886141-6903457  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
150  113-9637484-3998643  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
66   114-0291189-2266642  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
304  114-2588508-7212217  RC-VB-118150       1.0         -55.73   B0BHBNLNM1
287  114-5071876-1786635  RC-VB-118150       1.0         -55.73   B0BHBNLNM1

In [ ]:
suspect = ['B07HWTT4WY']
mask = refund_with_asin['(Child) ASIN'].isin(suspect)
cols = ['order id', 'sku',
        'quantity', 'product sales', '(Child) ASIN']
print(refund_with_asin.loc[mask, cols]
      .sort_values(['order id', 'sku'])
      .head(50))

                order id            sku  quantity  product sales (Child) ASIN
133  111-5513592-6005824  RB-LD-ReadKit       1.0          -99.0   B07HWTT4WY
223  111-7742931-8421864  RB-LD-ReadKit       1.0          -99.0   B07HWTT4WY
164  112-0486778-9727459  RB-LD-ReadKit       1.0          -99.0   B07HWTT4WY
188  112-2036937-7335436  RB-LD-ReadKit       1.0          -99.0   B07HWTT4WY
73   112-5226566-4768212  RB-LD-ReadKit       1.0          -79.2   B07HWTT4WY
286  112-8071577-2731410  RB-LD-ReadKit       1.0          -99.0   B07HWTT4WY
112  113-3940830-8207413  RB-LD-ReadKit       1.0          -99.0   B07HWTT4WY
158  113-4371067-0236269  RB-LD-ReadKit       1.0          -99.0   B07HWTT4WY
258  113-6868653-6262664  RB-LD-ReadKit       1.0          -99.0   B07HWTT4WY
130  114-1478922-8401855  RB-LD-ReadKit       1.0          -99.0   B07HWTT4WY
256  114-1972208-8045028  RB-LD-ReadKit       1.0          -99.0   B07HWTT4WY
154  114-3407995-2674649  RB-LD-ReadKit       1.0          -99.0

In [ ]:
# Step 6: Ensure all ASINs are included
all_asins = final_business_df[['(Child) ASIN']].drop_duplicates()

# Step 7: Group by ASIN
refund_grouped = (
    refund_with_asin
    .groupby('(Child) ASIN')[['quantity', 'product sales']]
    .sum()
    .rename(columns={
        'quantity': 'Total Returned Units',
        'product sales': 'Total Returned Sales'
    })
    .reset_index()
)

# Step 8: Merge with all ASINs
refund_agg = all_asins.merge(refund_grouped, on='(Child) ASIN', how='left')
refund_agg[['Total Returned Units', 'Total Returned Sales']] = refund_agg[
    ['Total Returned Units', 'Total Returned Sales']
].fillna(0)

In [ ]:
refund_agg.head(50)

,(Child) ASIN,Total Returned Units,Total Returned Sales
0,B01E46PJCE,22.0,-1269.40
1,B01E46WJIG,20.0,-1093.80
2,B01E44ZX9U,12.0,-726.90
3,B08MFNJSPG,12.0,-299.88
4,B0BHC99KGY,29.0,-1811.32
5,B0BHBNLNM1,17.0,-947.41
6,B0BHBW54WV,15.0,-1037.12
7,B0BL56DXNX,0.0,0.00
8,B07W7646SJ,15.0,-347.36
9,B004JR1YJQ,2.0,-89.96


In [ ]:
# Define output path
output_path = 'refund_summary_all_asins.xlsx'

# Export to Excel
refund_agg.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: refund_summary_all_asins.xlsx


In [ ]:
selling_econ_df.head()

,Amazon store,Start date,End date,Parent ASIN,ASIN,FNSKU,MSKU,Aged inventory surcharge per unit,Aged inventory surcharge quantity,Aged inventory surcharge total,...,Returns Processing Fee for Non-Apparel and Non-Shoes per unit,Returns Processing Fee for Non-Apparel and Non-Shoes quantity,Returns Processing Fee for Non-Apparel and Non-Shoes total,Returns processing fee for Apparel and Shoes per unit,Returns processing fee for Apparel and Shoes quantity,Returns processing fee for Apparel and Shoes total,Storage utilization surcharge per unit,Storage utilization surcharge quantity,Storage utilization surcharge total,FBA removal order fee per unit
0,US,2025-02-01,2025-02-28,B0DYL9FJXL,B00024JRC6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.64,0.0,0
1,US,2025-02-01,2025-02-28,B000CLYZZ8,B000CLYZZ8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,US,2025-02-01,2025-02-28,B000GCI9TG,B000GCI9TG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,US,2025-02-01,2025-02-28,B000HHJGEM,B000HHJGEM,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,US,2025-02-01,2025-02-28,B000HHJGFQ,B000HHJGFQ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
# Step 6: Fee column list
fee_columns = [
    "FBA disposal order fee per unit",
    "FBA removal order fee per unit",
    "Refund administration fee per unit",
    "Returns Processing Fee for Non-Apparel and Non-Shoes per unit",
    "Returns processing fee for Apparel and Shoes per unit"
]

# Ensure all fee columns exist and clean them
for col in fee_columns:
    if col not in selling_econ_df.columns:
        selling_econ_df[col] = 0.0
    else:
        selling_econ_df[col] = pd.to_numeric(selling_econ_df[col], errors='coerce').fillna(0)

# Step 7: Get most recent fee row per ASIN
selling_econ_df_sorted = selling_econ_df.sort_values(by=["ASIN", "End date"], ascending=[True, False])
latest_fees_df = selling_econ_df_sorted.drop_duplicates(subset="ASIN", keep="first")[["ASIN"] + fee_columns]

# Step 8: Merge refund data with fee data
refund_fees = refund_agg.merge(latest_fees_df, how="left", left_on="(Child) ASIN", right_on="ASIN")

# Step 9: Calculate total fees
for col in fee_columns:
    refund_fees[col + " (total)"] = refund_fees[col] * refund_fees["Total Returned Units"]

# Step 10: Calculate total return fee
refund_fees["Return Fee"] = refund_fees[[col + " (total)" for col in fee_columns]].sum(axis=1)

# Step 11: Final output with all details
total_fee_cols = [col + " (total)" for col in fee_columns]
final_refund_report = refund_fees[[
    "(Child) ASIN", "Total Returned Units", "Total Returned Sales", "Return Fee"
] + total_fee_cols]

In [ ]:
final_refund_report.head(20)

,(Child) ASIN,Total Returned Units,Total Returned Sales,Return Fee,FBA disposal order fee per unit (total),FBA removal order fee per unit (total),Refund administration fee per unit (total),Returns Processing Fee for Non-Apparel and Non-Shoes per unit (total),Returns processing fee for Apparel and Shoes per unit (total)
0,B01E46PJCE,22.0,-1269.40,38.72,0.00,0.0,38.72,0.0,0.0
1,B01E46WJIG,20.0,-1093.80,51.80,20.80,0.0,31.00,0.0,0.0
2,B01E44ZX9U,12.0,-726.90,23.76,0.00,0.0,23.76,0.0,0.0
3,B08MFNJSPG,12.0,-299.88,21.48,12.48,0.0,9.00,0.0,0.0
4,B0BHC99KGY,29.0,-1811.32,53.36,0.00,0.0,53.36,0.0,0.0
5,B0BHBNLNM1,17.0,-947.41,26.18,0.00,0.0,26.18,0.0,0.0
6,B0BHBW54WV,15.0,-1037.12,29.25,0.00,0.0,29.25,0.0,0.0
7,B0BL56DXNX,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0
8,B07W7646SJ,15.0,-347.36,26.70,15.60,0.0,11.10,0.0,0.0
9,B004JR1YJQ,2.0,-89.96,5.76,3.06,0.0,2.70,0.0,0.0


In [ ]:
# Define output file path
output_path = 'detailed_refund_fee_report.xlsx'

# Export to Excel
final_refund_report.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: detailed_refund_fee_report.xlsx


In [ ]:
# Step 1: Filter return data for CUSTOMER_DAMAGED or DEFECTIVE
filtered_return_df = return_df[
    return_df['detailed-disposition'].isin(['CUSTOMER_DAMAGED', 'DEFECTIVE'])
].copy()

# Step 2: Clean quantity column
filtered_return_df['quantity'] = pd.to_numeric(filtered_return_df['quantity'], errors='coerce').fillna(0)

# Step 3: Group by ASIN and aggregate return units
return_agg = filtered_return_df.groupby('asin')['quantity'].sum().reset_index()
return_agg.rename(columns={'quantity': 'Defective/Damaged Returned Units'}, inplace=True)

# Step 4: Clean and standardize ASINs
return_agg['asin'] = return_agg['asin'].astype(str).str.strip().str.upper()
unit_financial_df['ASIN'] = unit_financial_df['ASIN'].astype(str).str.strip().str.upper()

# Step 5: Handle Current Wholesale Price
if 'Current Wholesale Price' not in unit_financial_df.columns:
    agg_df['Current Wholesale Price'] = 0.0
else:
    agg_df['Current Wholesale Price'] = pd.to_numeric(
        agg_df['Current Wholesale Price'], errors='coerce'
    ).fillna(0)

# Step 6: Prepare product cost DataFrame
product_cost_df = unit_financial_df[['ASIN', 'Current Wholesale Price']].drop_duplicates(subset='ASIN')

In [ ]:
filtered_df_test2 = product_cost_df[product_cost_df['ASIN'] == 'B001ODZ5BQ']
filtered_price2 = filtered_df_test2['Current Wholesale Price']
print(filtered_price2)

777    29.26
Name: Current Wholesale Price, dtype: float64


In [ ]:
product_cost_df['Current Wholesale Price'] = product_cost_df['Current Wholesale Price'].round(2)
product_cost_df.head()

,ASIN,Current Wholesale Price
0,B07PH9Z1VG,25.97
1,B07QQKKVXQ,55.32
2,0,111.72
3,B07PYF9RZ6,23.94
5,B07QFBL57H,24.53


In [ ]:
# Step 7: Clean ASINs in shipping/label fees DataFrame
recent_values_df['ASIN'] = recent_values_df['ASIN'].astype(str).str.strip().str.upper()

for col in ['Shipping Fee', 'Label Fee']:
    if col not in recent_values_df.columns:
        recent_values_df[col] = 0.0
    else:
        recent_values_df[col] = pd.to_numeric(recent_values_df[col], errors='coerce').fillna(0)

shipping_label_df = recent_values_df[['ASIN', 'Shipping Fee', 'Label Fee']]

In [ ]:
# Step 8: Merge with Product Cost
merged = return_agg.merge(product_cost_df, how='left', left_on='asin', right_on='ASIN')

# Step 9: Merge with Shipping & Label Fees
merged = merged.merge(
    shipping_label_df,
    how='left',
    left_on='asin',
    right_on='ASIN',
    suffixes=('', '_drop')  # if overlapping columns
)


In [ ]:
filtered_df_test3 = merged[merged['asin'] == 'B001ODZ5BQ']
filtered_price3 = filtered_df_test3[['Current Wholesale Price', 'Defective/Damaged Returned Units']]
filtered_price4 = filtered_df_test3['Current Wholesale Price']
print(filtered_price3)
print(filtered_price4)


   Current Wholesale Price  Defective/Damaged Returned Units
0                    29.26                                 1
0    29.26
Name: Current Wholesale Price, dtype: float64


In [ ]:
# Step 12: Compute return cost components
merged['Product Cost of Return'] = merged['Current Wholesale Price'] * merged['Defective/Damaged Returned Units']
merged['Return Shipping Fee'] = merged['Shipping Fee'] * merged['Defective/Damaged Returned Units']
merged['Return Label Fee'] = merged['Label Fee'] * merged['Defective/Damaged Returned Units']

# Step 13: Final output
final_return_cost_report = merged[[
    'asin',
    'Defective/Damaged Returned Units',
    'Current Wholesale Price',
    'Product Cost of Return',
    'Return Shipping Fee',
    'Return Label Fee'
]]


In [ ]:
final_return_cost_report.head(20)

,asin,Defective/Damaged Returned Units,Current Wholesale Price,Product Cost of Return,Return Shipping Fee,Return Label Fee
0,B001ODZ5BQ,1,29.26,29.26,0.00,0.00
1,B004JR1YJQ,1,22.93,22.93,0.92,0.60
2,B008DXYTGW,1,7.89,7.89,0.00,0.59
3,B01E46WJIG,1,36.39,36.39,0.00,0.00
4,B074P6C27C,4,13.79,55.16,0.00,2.24
5,B07CTY1K6Q,1,23.40,23.40,0.24,0.00
6,B07CZ1DJRD,1,24.73,24.73,0.50,0.00
7,B07HWTT4WY,11,45.00,495.00,17.16,6.05
8,B07W7646SJ,28,12.71,355.88,0.00,0.00
9,B07X7HBL5N,1,26.21,26.21,0.17,0.00


In [ ]:
# Define output path
output_path = 'return_cost_breakdown_report.xlsx'

# Export to Excel
final_return_cost_report.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: return_cost_breakdown_report.xlsx


In [ ]:
# Step 1: Load extra transactions (June to Feb)
june_to_feb_df = pd.read_csv("/content/2024Nov1-2025Feb28CustomUnifiedTransaction.xlsx.csv")

# Step 2: Clean 'date/time' by removing timezone like 'PST', 'PDT' etc.
june_to_feb_df['date/time'] = june_to_feb_df['date/time'].astype(str).str.replace(r'\s+[A-Z]{2,4}$', '', regex=True)

# Step 3: Convert cleaned date/time to datetime
june_to_feb_df['date/time'] = pd.to_datetime(june_to_feb_df['date/time'], errors='coerce')

# Step 4: Sort by date descending
june_to_feb_df = june_to_feb_df.sort_values(by='date/time', ascending=False).reset_index(drop=True)


/tmp/ipython-input-60-3444330709.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  june_to_feb_df = pd.read_csv("/content/2024Nov1-2025Feb28CustomUnifiedTransaction.xlsx.csv")
/tmp/ipython-input-60-3444330709.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  june_to_feb_df['date/time'] = pd.to_datetime(june_to_feb_df['date/time'], errors='coerce')


In [ ]:
june_to_feb_df.head()

,date/time,settlement id,type,order id,sku,description,quantity,marketplace,account type,fulfillment,...,Regulatory Fee,Tax On Regulatory Fee,promotional rebates,promotional rebates tax,marketplace withheld tax,selling fees,fba fees,other transaction fees,other,total
0,2025-02-28 23:59:56,22960216471,Order,113-4269721-2654628,RC-PBL-118061,Yummy Combs Dog Dental Treats - Vet VOHC Appro...,1.0,amazon.com,Standard Orders,Amazon,...,0,0,0.00,0,-3.17,-7.5,-6.05,0.0,0,36.43
1,2025-02-28 23:58:10,22960216471,Order,112-3636250-3545827,RC-PBL-118064,Yummy Combs Dog Dental Treats - Vet VOHC Appro...,1.0,amazon.com,Standard Orders,Amazon,...,0,0,-2.99,0,0.00,-7.5,-6.05,0.0,0,36.43
2,2025-02-28 23:58:06,22960216471,Order,111-0872957-3076267,RC-VB-062095-1,MOVOFLEX Soft Chews Hip and Joint Support Supp...,1.0,amazon.com,Standard Orders,Amazon,...,0,0,-2.99,0,-6.57,-9.52,-4.51,0.0,0,49.44
3,2025-02-28 23:56:44,22960216471,Order,111-8288533-3004247,RC-V1-600170,VetOne Dog Shampoo and Conditioner - VetraSeb ...,1.0,amazon.com,Standard Orders,Amazon,...,0,0,0.00,0,-1.57,-2.7,-4.15,0.0,0,11.14
4,2025-02-28 23:54:07,22960216471,Order,113-3984455-7233040,RC-PBL-118060,Yummy Combs Dog Dental Treats - Vet VOHC Appro...,1.0,amazon.com,Standard Orders,Amazon,...,0,0,-2.50,0,0.00,0,-6.05,0.0,0,43.44


In [ ]:
# Step 1: Filter return_df for DEFECTIVE rows
defective_returns = return_df[
    return_df['detailed-disposition'].isin(['DEFECTIVE'])
].copy()
defective_returns['quantity'] = pd.to_numeric(defective_returns['quantity'], errors='coerce').fillna(0)

# Step 2: Group by ASIN to get total defective quantity
defective_agg = defective_returns.groupby('asin')['quantity'].sum().reset_index()
defective_agg.rename(columns={'quantity': 'Total DEFECTIVE Qty'}, inplace=True)

In [ ]:
len(defective_agg)

10

In [ ]:
# STEP 3: Extract Refunds and Orders separately from june_to_feb_df
refunds = transaction_df[transaction_df['type'] == 'Refund'].copy()
orders = june_to_feb_df[june_to_feb_df['type'] == 'Order'].copy()

# Ensure business_df has unique SKU-to-ASIN mapping
business_unique = final_business_df[['SKU', '(Child) ASIN']].drop_duplicates(subset='SKU')

# STEP 4: Keep the most recent 'Order' for each order id
orders_recent = orders.drop_duplicates(subset=['order id', 'sku'], keep='first').copy()

# STEP 5: Clean fba fees in recent orders
orders_recent['fba fees'] = pd.to_numeric(orders_recent['fba fees'], errors='coerce').fillna(0)

# STEP 6: Attach FBA fees to refund records based on matching 'order id'
refunds = refunds.drop(columns=['fba fees'], errors='ignore')  # just in case
refunds_with_fees = refunds.merge(
    orders_recent[['order id', 'sku', 'fba fees']],
    how='left',
    on=['order id', 'sku']
)

# STEP 7: Add ASIN using SKU mapping from business_df
refunds_with_fees['sku'] = refunds_with_fees['sku'].astype(str).str.strip()


# Step B: Merge refunds_with_fees into sku_asin_map (to get all ASINs from business_df)
refunds_with_fees_asin = refunds_with_fees.merge(
    business_unique,
    how='left',
    left_on='sku',
    right_on='SKU'
)

refunds_with_fees_asin.rename(columns={'(Child) ASIN': 'asin'}, inplace=True)

# STEP 8: Drop rows with missing ASINs (optional)
refunds_with_fees_asin = refunds_with_fees_asin.dropna(subset=['asin'])

# ✅ NEW STEP 9: Drop duplicates to avoid double-counting fees from multiple refunds for same order
# refunds_unique = refunds_with_fees_asin.drop_duplicates(subset=['order id', 'asin'])
# ✅ Updated deduplication: consider order id + SKU + ASIN
refunds_unique = refunds_with_fees_asin.drop_duplicates(subset=['order id', 'sku', 'asin'])


# STEP 9: Sum FBA fees by ASIN
return_fulfillment_fee = refunds_with_fees_asin.groupby('asin')['fba fees'].sum().reset_index()
return_fulfillment_fee.rename(columns={'fba fees': 'Return Fulfillment Fee'}, inplace=True)




In [ ]:
return_fulfillment_fee.head()

,asin,Return Fulfillment Fee
0,B004JR1YJQ,-9.82
1,B005JTY9RW,-14.75
2,B008DXYTGW,-4.43
3,B00AT6FY6A,-11.87
4,B00C8RJ3BA,-5.37


In [ ]:
len(return_fulfillment_fee)

85

In [ ]:
# Check Total DEFECTIVE Qty for specific ASIN
asin_to_check = 'B09DDBTYGV'
result_row = return_fulfillment_fee[return_fulfillment_fee['asin'] == asin_to_check]

# Display the value
if not result_row.empty:
    print(f"Total DEFECTIVE Qty for ASIN {asin_to_check}: {result_row['Return Fulfillment Fee'].values[0]}")
else:
    print(f"ASIN {asin_to_check} not found in final_result.")


Total DEFECTIVE Qty for ASIN B09DDBTYGV: -5.52


In [ ]:
# STEP 10: Merge with defective quantity table
final_result = pd.merge(return_fulfillment_fee, defective_agg, on='asin', how='outer')
final_result['Return Fulfillment Fee'] = final_result['Return Fulfillment Fee'].fillna(0)
final_result['Total DEFECTIVE Qty'] = final_result['Total DEFECTIVE Qty'].fillna(0)

In [ ]:
# Step 1: Filter all refund rows for the ASIN
asin_debug_df = refunds_with_fees_asin[refunds_with_fees_asin['asin'] == 'B0CF8248YC'].copy()

# Step 2: Check how many unique (order id, sku) pairs are associated with this ASIN
print("🔍 Unique (order id, sku) pairs for this ASIN:")
print(asin_debug_df[['order id', 'sku']].drop_duplicates().shape[0])

# Step 3: Group by order ID to see how many times each order appears
print("\n📊 Refund rows per order id:")
print(asin_debug_df.groupby('order id').size().sort_values(ascending=False).head(50))

# Step 4: Inspect full details for each refund + FBA fee applied
print("\n📋 Refund rows with FBA fees:")
print(asin_debug_df[['order id', 'sku', 'asin', 'fba fees', 'quantity']].sort_values(by='order id'))

# Step 5: Drop duplicates using (order id, sku) for better accuracy
asin_unique_debug = asin_debug_df.drop_duplicates(subset=['order id', 'sku'])

print("\n✅ Unique refund rows (order id + sku):")
print(asin_unique_debug[['order id', 'sku', 'fba fees']])

# Step 6: Compare total FBA fees (raw vs. deduplicated)
total_raw = asin_debug_df['fba fees'].sum()
total_unique = asin_unique_debug['fba fees'].sum()

print(f"\n💰 Total FBA Fees (raw): ${total_raw:.2f}")
print(f"💰 Total FBA Fees (unique order+sku): ${total_unique:.2f}")


🔍 Unique (order id, sku) pairs for this ASIN:
4

📊 Refund rows per order id:
order id
111-5022365-5539411    1
111-8287634-7767420    1
112-5979805-0300247    1
114-4938797-6765867    1
dtype: int64

📋 Refund rows with FBA fees:
                order id          sku        asin  fba fees  quantity
183  111-5022365-5539411  RB-WRA-GT04  B0CF8248YC     -3.15       1.0
102  111-8287634-7767420  RB-WRA-GT04  B0CF8248YC     -3.15       1.0
15   112-5979805-0300247  RB-WRA-GT04  B0CF8248YC     -3.15       1.0
36   114-4938797-6765867  RB-WRA-GT04  B0CF8248YC     -3.15       1.0

✅ Unique refund rows (order id + sku):
                order id          sku  fba fees
15   112-5979805-0300247  RB-WRA-GT04     -3.15
36   114-4938797-6765867  RB-WRA-GT04     -3.15
102  111-8287634-7767420  RB-WRA-GT04     -3.15
183  111-5022365-5539411  RB-WRA-GT04     -3.15

💰 Total FBA Fees (raw): $-12.60
💰 Total FBA Fees (unique order+sku): $-12.60


In [ ]:
# Step 1: Filter all refund rows for the ASIN
asin_debug_df = refunds_with_fees_asin[refunds_with_fees_asin['asin'] == 'B09DDBTYGV'].copy()

# Step 2: Check how many unique (order id, sku) pairs are associated with this ASIN
print("🔍 Unique (order id, sku) pairs for this ASIN:")
print(asin_debug_df[['order id', 'sku']].drop_duplicates().shape[0])

# Step 3: Group by order ID to see how many times each order appears
print("\n Refund rows per order id:")
print(asin_debug_df.groupby('order id').size().sort_values(ascending=False).head(50))

# Step 4: Inspect full details for each refund + FBA fee applied
print("\n Refund rows with FBA fees:")
print(asin_debug_df[['order id', 'sku', 'asin', 'fba fees', 'quantity']].sort_values(by='order id'))

# Step 5: Check how many of these rows remain after deduplication
asin_unique_debug = asin_debug_df.drop_duplicates(subset=['order id', 'asin'])
print("\n Unique refund rows (no double-counting):")
print(asin_unique_debug[['order id', 'fba fees']])

# Step 6: Total FBA fee from raw vs. unique
total_raw = asin_debug_df['fba fees'].sum()
total_unique = asin_unique_debug['fba fees'].sum()

print(f"\n Total FBA Fees (raw): ${total_raw:.2f}")
print(f" Total FBA Fees (unique orders): ${total_unique:.2f}")

🔍 Unique (order id, sku) pairs for this ASIN:
1

 Refund rows per order id:
order id
114-8768616-1175447    1
dtype: int64

 Refund rows with FBA fees:
                order id            sku        asin  fba fees  quantity
251  114-8768616-1175447  RB-KB-KDTL105  B09DDBTYGV     -5.52       1.0

 Unique refund rows (no double-counting):
                order id  fba fees
251  114-8768616-1175447     -5.52

 Total FBA Fees (raw): $-5.52
 Total FBA Fees (unique orders): $-5.52


In [ ]:
# Check Total DEFECTIVE Qty for specific ASIN
asin_to_check = 'B07CZ1DJRD'
result_row = final_result[final_result['asin'] == asin_to_check]

# Display the value
if not result_row.empty:
    print(f"Total DEFECTIVE Qty for ASIN {asin_to_check}: {result_row['Total DEFECTIVE Qty'].values[0]}")
else:
    print(f"ASIN {asin_to_check} not found in final_result.")


Total DEFECTIVE Qty for ASIN B07CZ1DJRD: 1.0


In [ ]:
final_result.head()

,asin,Return Fulfillment Fee,Total DEFECTIVE Qty
0,B004JR1YJQ,-9.82,1.0
1,B005JTY9RW,-14.75,0.0
2,B008DXYTGW,-4.43,1.0
3,B00AT6FY6A,-11.87,0.0
4,B00C8RJ3BA,-5.37,0.0


In [ ]:
# Define output path
output_path = 'return_fulfillment_fee_report.xlsx'

# Export to Excel
final_result.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: return_fulfillment_fee_report.xlsx


In [ ]:
# # Step 0: Create master list of all ASINs from business_df
# all_asins_master = unit_financial_df[['ASIN']].drop_duplicates().rename(columns={'ASIN': 'asin'})

In [ ]:
# Step 12: Normalize refund report (rename column)
refund_base = final_refund_report.rename(columns={"(Child) ASIN": "asin"})

# Step 13: Now merge refund report + return cost + final_result
merged_costs = refund_base.merge(final_return_cost_report, on='asin', how='outer')
merged_costs = merged_costs.merge(final_result, on='asin', how='outer')

# Step 14: Fill missing numeric columns with 0
numeric_cols = merged_costs.select_dtypes(include='number').columns
merged_costs[numeric_cols] = merged_costs[numeric_cols].fillna(0)

In [ ]:
# # Step 1: Normalize refund report to use 'asin' consistently
# refund_base = final_refund_report.rename(columns={"(Child) ASIN": "asin"})

# # Step 2: LEFT merge all reports step-by-step
# merged_costs = all_asins_master.merge(refund_base, on='asin', how='left')
# merged_costs = merged_costs.merge(final_return_cost_report, on='asin', how='left')
# merged_costs = merged_costs.merge(final_result, on='asin', how='left')

# # Step 3: Fill NaNs for numeric columns
# numeric_cols = merged_costs.select_dtypes(include='number').columns
# merged_costs[numeric_cols] = merged_costs[numeric_cols].fillna(0)

# Step 4: Normalize cost columns to be negative if needed
cost_columns = [
    'Return Fee',
    'FBA disposal order fee per unit (total)',
    'FBA removal order fee per unit (total)',
    'Refund administration fee per unit (total)',
    'Returns Processing Fee for Non-Apparel and Non-Shoes per unit (total)',
    'Returns processing fee for Apparel and Shoes per unit (total)',
    'Product Cost of Return',
    'Return Shipping Fee',
    'Return Label Fee',
    'Return Fulfillment Fee'
]

for col in cost_columns:
    if col in merged_costs.columns:
        merged_costs[col] = merged_costs[col].apply(
            lambda x: x if x <= 0 else -x
        )


# Step 5: Final result
final_merged = merged_costs.reset_index(drop=True)
final_merged['Net Cost of Return'] = (
    final_merged['Return Fee'] +
    final_merged['Product Cost of Return'] +
    final_merged['Return Shipping Fee'] +
    final_merged['Return Label Fee'] +
    final_merged['Return Fulfillment Fee']
)

In [ ]:
# Step 1: Merge using (Child) ASIN from final_df with asin in final_merged
final_merged = final_merged.merge(
    final_df[['ASIN', 'quantity']],
    how='left',
    left_on='asin',           # match final_merged.asin
    right_on='ASIN'    # match final_df['(Child) ASIN']
)

# Step 2: Rename 'quantity' to 'quantity sold'
final_merged = final_merged.rename(columns={'quantity': 'quantity sold'})

# Step 3: (Optional) Fill missing quantity sold with 0
final_merged['quantity sold'] = final_merged['quantity sold'].fillna(0)

# Step 4: Drop the extra (Child) ASIN column if you want (optional clean up)
# final_merged = final_merged.drop(columns=['quantity sold'])

# Step 5: Reorder columns
new_column_order = [
    'asin',
    'quantity sold',
    'Total Returned Units',
    'Defective/Damaged Returned Units',
    'Total DEFECTIVE Qty',
    'Total Returned Sales',
    'Return Fulfillment Fee',
    'Product Cost of Return',
    'Return Shipping Fee',
    'Return Label Fee',
    'Return Fee',
    'Net Cost of Return'
]

final_merged = final_merged[new_column_order]

# Step 6: Display to confirm
final_merged


,asin,quantity sold,Total Returned Units,Defective/Damaged Returned Units,Total DEFECTIVE Qty,Total Returned Sales,Return Fulfillment Fee,Product Cost of Return,Return Shipping Fee,Return Label Fee,Return Fee,Net Cost of Return
0,B00024JRC6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,B000CLYZZ8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,B000HHJGEM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B000HHLJ24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B000HHLRGW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
635,B0DYFX5KRT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
636,B0DYFX6SGF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
637,B0F22K57MY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
638,B0F22KW4C1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Safe copy
final_merged_with_total = final_merged.copy()

# Make columns unique manually without deprecated function
def make_columns_unique(columns):
    seen = {}
    new_cols = []
    for col in columns:
        if col in seen:
            seen[col] += 1
            new_cols.append(f"{col}.{seen[col]}")
        else:
            seen[col] = 0
            new_cols.append(col)
    return new_cols

# Apply manual deduplication
final_merged_with_total.columns = make_columns_unique(final_merged_with_total.columns)

# Step 1: Sum all numeric columns
total_row = final_merged_with_total.select_dtypes(include='number').sum()

# Step 2: Set 'asin' to 'Total'
total_row['asin'] = 'Total'

# Step 3: Convert Series to DataFrame
total_row_df = pd.DataFrame([total_row])

# Step 4: Append total row
final_merged_with_total = pd.concat([final_merged_with_total, total_row_df], ignore_index=True)

# Step 5: Reorder columns if needed
cols = ['asin'] + [col for col in final_merged_with_total.columns if col != 'asin']
final_merged_with_total = final_merged_with_total[cols]

# ✅ Now your final_merged_with_total is perfect!
final_merged_with_total


,asin,quantity sold,Total Returned Units,Defective/Damaged Returned Units,Total DEFECTIVE Qty,Total Returned Sales,Return Fulfillment Fee,Product Cost of Return,Return Shipping Fee,Return Label Fee,Return Fee,Net Cost of Return
0,B00024JRC6,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00
1,B000CLYZZ8,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00
2,B000HHJGEM,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00
3,B000HHLJ24,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00
4,B000HHLRGW,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
636,B0DYFX6SGF,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00
637,B0F22K57MY,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00
638,B0F22KW4C1,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00
639,B0F2GTFZ7V,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00


In [ ]:
# Check for a specific ASIN in final_merged
asin_to_check = 'B07HWTT4WY'

result = final_merged_with_total[final_merged_with_total['asin'] == asin_to_check]

# Display the result
if not result.empty:
    print(result)
else:
    print(f"❌ ASIN {asin_to_check} not found in final_merged.")


           asin  quantity sold  Total Returned Units  \
242  B07HWTT4WY          166.0                  14.0   

     Defective/Damaged Returned Units  Total DEFECTIVE Qty  \
242                              11.0                  0.0   

     Total Returned Sales  Return Fulfillment Fee  Product Cost of Return  \
242               -1366.2                 -119.92                  -495.0   

     Return Shipping Fee  Return Label Fee  Return Fee  Net Cost of Return  
242               -17.16             -6.05     -140.84             -778.97  


In [ ]:
# Define the output file path
output_path = 'full_detailed_return_report.xlsx'

# Export full_report to Excel
final_merged_with_total.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: full_detailed_return_report.xlsx


In [ ]:
import numpy as np
# Step 1: Merge final_df and final_merged on ASIN
final_df_merged = final_df.merge(
    final_merged[['asin', 'Net Cost of Return']],  # Only need asin and Net Cost of Return
    left_on='ASIN',  # Your final_df uses (Child) ASIN
    right_on='asin',
    how='left'
)


# Step 2: Rename necessary columns
final_df_merged = final_df_merged.rename(columns={
    'Net Cost of Return': 'cost of returns',
    'Shipping Fee': 'shipping',
    'Label Fee': 'label',
    'total storage cost': 'storage',
    'Allocatable Fees': 'Allocated fees (Premium Services Fee + Subscription)'
})


# Step 3: Drop extra 'asin' column (optional clean)
final_df_merged = final_df_merged.drop(columns=['asin'])

# Step 4: (Optional) Fill missing cost of returns with 0
final_df_merged['cost of returns'] = final_df_merged['cost of returns'].fillna(0)


cost_fields = [
    'Product Cost', 'selling fees', 'fba fees', 'other transaction fees',
    'shipping', 'label', 'storage',
    'Allocated fees (Premium Services Fee + Subscription)', 'cost of returns'
]

# Convert all costs to negative
for col in cost_fields:
    final_df_merged[col] = final_df_merged[col].apply(lambda x: -abs(x) if pd.notnull(x) else 0)

# Your current formula then works
final_df_merged['Gross Profit'] = (
    final_df_merged['product sales']
    + final_df_merged['Product Cost']
    + final_df_merged['selling fees']
    + final_df_merged['fba fees']
    + final_df_merged['other transaction fees']
    + final_df_merged['shipping']
    + final_df_merged['label']
    + final_df_merged['storage']
    + final_df_merged['Allocated fees (Premium Services Fee + Subscription)']
    + final_df_merged['cost of returns']
)


final_df_merged['Gross Margin'] = (
    final_df_merged['Gross Profit']
    / final_df_merged['product sales']
)
# Fill all NaN values with 0
final_df_merged = final_df_merged.replace([np.inf, -np.inf], np.nan).fillna(0)

final_df_merged['Gross Margin'] = (final_df_merged['Gross Margin'] * 100).round(2).astype(str) + '%'

In [ ]:
final_df_merged

,ASIN,product sales,selling fees,fba fees,other transaction fees,quantity,Allocated fees (Premium Services Fee + Subscription),Gross Profit,Current Wholesale Price,Product Cost,Featured Offer (Buy Box) Percentage,Sessions - Total,Unit Session Percentage,storage,shipping,label,cost of returns,Gross Margin
0,B01E46PJCE,136654.77,-19274.58,-9299.90,-0.0,2153.0,-580.96,16382.31,42.24,-90942.72,92.84%,"59,330",61.97%,-40.16,-0.0,-0.0,-134.14,11.99%
1,B01E46WJIG,87232.27,-12359.24,-6484.74,-0.0,1595.0,-370.85,9793.34,36.39,-58042.05,94.20%,"55,537",51.22%,-14.26,-0.0,-0.0,-167.79,11.23%
2,B01E44ZX9U,93948.52,-13218.54,-6090.84,-0.0,1293.0,-399.40,11558.05,48.39,-62568.27,92.75%,"43,542",55.70%,-33.76,-0.0,-0.0,-79.66,12.3%
3,B08MFNJSPG,667.23,-100.11,-112.05,-0.0,27.0,-2.84,-88.47,12.71,-343.17,96.35%,"195,716",10.03%,-9.34,-0.0,-0.0,-188.19,-13.26%
4,B0BHC99KGY,112884.05,-16132.85,-7549.79,-0.0,1745.0,-479.90,12227.54,43.72,-76291.40,97.37%,"54,908",33.75%,-26.02,-0.0,-0.0,-176.55,10.83%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,B0C8WWZQKR,0.00,-0.00,-0.00,-0.0,0.0,-0.00,0.00,110.00,-0.00,95.24%,54,1.85%,0.00,-0.0,-0.0,-0.00,0.0%
636,B0C8X5GF7D,0.00,-0.00,-0.00,-0.0,0.0,-0.00,0.00,110.00,-0.00,100.00%,9,11.11%,0.00,-0.0,-0.0,-0.00,0.0%
637,B0C8X67WC1,0.00,-0.00,-0.00,-0.0,0.0,-0.00,0.00,110.00,-0.00,100.00%,38,2.63%,0.00,-0.0,-0.0,-0.00,0.0%
638,B0CLYX3YVS,0.00,-0.00,-0.00,-0.0,0.0,-0.00,0.00,0.00,-0.00,67.72%,162,0.62%,0.00,-0.0,-0.0,-0.00,0.0%


In [ ]:
asin_to_check = 'B0DGB65B4M'

asin_row = final_df_merged[final_df_merged['ASIN'] == asin_to_check]

# Show the relevant columns
asin_row


,ASIN,product sales,selling fees,fba fees,other transaction fees,quantity,Allocated fees (Premium Services Fee + Subscription),Gross Profit,Current Wholesale Price,Product Cost,Featured Offer (Buy Box) Percentage,Sessions - Total,Unit Session Percentage,storage,shipping,label,cost of returns,Gross Margin
188,B0DGB65B4M,1620.0,-194.4,-0.0,-0.0,20.0,-6.89,782.71,31.8,-636.0,98.39%,"5,222",1.97%,0.0,-0.0,-0.0,-0.0,48.32%


In [ ]:
# Safe copy
final_df_merged_with_total = final_df_merged.copy()

# Step 1: Clean percentage columns
percentage_columns = [
    'Featured Offer (Buy Box) Percentage',
    'Unit Session Percentage',
    'Gross Margin'
]

for col in percentage_columns:
    final_df_merged_with_total[col] = final_df_merged_with_total[col].str.rstrip('%').astype(float)

# Step 2: Clean 'Sessions - Total' (remove commas and convert to numeric)
final_df_merged_with_total['Sessions - Total'] = (
    final_df_merged_with_total['Sessions - Total']
    .str.replace(',', '', regex=False)
    .astype(float)
)

# Step 3: Sum all numeric columns
total_row = final_df_merged_with_total.select_dtypes(include='number').sum()

# Step 4: Special calculations for total_row

# (1) Gross Profit: recalculate using your formula
total_row['Gross Profit'] = (
    total_row['product sales']
    + total_row['Product Cost']
    + total_row['selling fees']
    + total_row['fba fees']
    + total_row['other transaction fees']
    + total_row['shipping']
    + total_row['label']
    + total_row['storage']
    + total_row['Allocated fees (Premium Services Fee + Subscription)']
    + total_row['cost of returns']
)

# (2) Gross Margin: recalculate
if total_row['product sales'] != 0:
    total_row['Gross Margin'] = total_row['Gross Profit'] / total_row['product sales']
else:
    total_row['Gross Margin'] = 0

# (3) Featured Offer (Buy Box) % and Unit Session %: use average
total_row['Featured Offer (Buy Box) Percentage'] = final_df_merged_with_total['Featured Offer (Buy Box) Percentage'].mean()
total_row['Unit Session Percentage'] = final_df_merged_with_total['Unit Session Percentage'].mean()

# (4) Sessions - Total is already summed correctly

# (5) Add ASIN as Total
total_row['ASIN'] = 'Total'

# Step 5: Append total row
final_df_merged_with_total = pd.concat([
    final_df_merged_with_total,
    pd.DataFrame([total_row])
], ignore_index=True)

# Step 6: Format the columns properly

# (1) Format quantity and Sessions - Total with comma (no decimal)
final_df_merged_with_total['quantity'] = final_df_merged_with_total['quantity'].astype(int).apply(lambda x: f"{x:,}")
final_df_merged_with_total['Sessions - Total'] = final_df_merged_with_total['Sessions - Total'].astype(int).apply(lambda x: f"{x:,}")

# (2) Format money fields with $ and 2 decimals + comma
money_columns = [
    'product sales',
    'Product Cost',
    'selling fees',
    'fba fees',
    'other transaction fees',
    'shipping',
    'label',
    'storage',
    'Allocated fees (Premium Services Fee + Subscription)',
    'cost of returns',
    'Gross Profit'
]

for col in money_columns:
    final_df_merged_with_total[col] = final_df_merged_with_total[col].apply(
        lambda x: f"${float(str(x).replace('$', '').replace(',', '')):,.2f}"
    )

# (3) Format percentage fields
# Multiply only the Total row's Gross Margin by 100
final_df_merged_with_total.loc[
    final_df_merged_with_total['ASIN'] == 'Total', 'Gross Margin'
] *= 100

for col in percentage_columns:
    final_df_merged_with_total[col] = final_df_merged_with_total[col].apply(
        lambda x: f"{round(float(x), 2)}%"
    )

# Step 7: Rename columns to match your screenshot
rename_mapping = {
    'product sales': 'Amazon Top-line Sales (ATS)',
    'Product Cost': 'Product Cost',
    'selling fees': 'Referral Fee',
    'fba fees': 'FBA Fulfillment Fee',
    'other transaction fees': 'other transaction fees',
    'shipping': 'Shipping/Kitting Fees',
    'label': 'Labeling/Polybagging Fees',
    'storage': 'Storage Fees',
    'Allocated fees (Premium Services Fee + Subscription)': 'Allocated fees (Premium Services Fee + Subscription)',
    'cost of returns': 'Cost of Returns',
    'Gross Profit': 'Gross Profit',
    'Gross Margin': 'Gross Margin'
}
final_df_merged_with_total.rename(columns=rename_mapping, inplace=True)

# Step 8: Reorder columns to match your screenshot
cols = [
    'ASIN',
    'quantity',
    'Amazon Top-line Sales (ATS)',
    'Product Cost',
    'Referral Fee',
    'FBA Fulfillment Fee',
    'other transaction fees',
    'Shipping/Kitting Fees',
    'Labeling/Polybagging Fees',
    'Storage Fees',
    'Allocated fees (Premium Services Fee + Subscription)',
    'Cost of Returns',
    'Gross Profit',
    'Gross Margin',
    'Featured Offer (Buy Box) Percentage',
    'Sessions - Total',
    'Unit Session Percentage'
]

final_df_merged_with_total = final_df_merged_with_total[cols]

# ✅ Done: final_df_merged_with_total is now fully cleaned, formatted, totaled, and renamed
final_df_merged_with_total


,ASIN,quantity,Amazon Top-line Sales (ATS),Product Cost,Referral Fee,FBA Fulfillment Fee,other transaction fees,Shipping/Kitting Fees,Labeling/Polybagging Fees,Storage Fees,Allocated fees (Premium Services Fee + Subscription),Cost of Returns,Gross Profit,Gross Margin,Featured Offer (Buy Box) Percentage,Sessions - Total,Unit Session Percentage
0,B01E46PJCE,"2,153","$136,654.77","$-90,942.72","$-19,274.58","$-9,299.90",$-0.00,$-0.00,$-0.00,$-40.16,$-580.96,$-134.14,"$16,382.31",11.99%,92.84%,"59,330",61.97%
1,B01E46WJIG,"1,595","$87,232.27","$-58,042.05","$-12,359.24","$-6,484.74",$-0.00,$-0.00,$-0.00,$-14.26,$-370.85,$-167.79,"$9,793.34",11.23%,94.2%,"55,537",51.22%
2,B01E44ZX9U,"1,293","$93,948.52","$-62,568.27","$-13,218.54","$-6,090.84",$-0.00,$-0.00,$-0.00,$-33.76,$-399.40,$-79.66,"$11,558.05",12.3%,92.75%,"43,542",55.7%
3,B08MFNJSPG,27,$667.23,$-343.17,$-100.11,$-112.05,$-0.00,$-0.00,$-0.00,$-9.34,$-2.84,$-188.19,$-88.47,-13.26%,96.35%,"195,716",10.03%
4,B0BHC99KGY,"1,745","$112,884.05","$-76,291.40","$-16,132.85","$-7,549.79",$-0.00,$-0.00,$-0.00,$-26.02,$-479.90,$-176.55,"$12,227.54",10.83%,97.37%,"54,908",33.75%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,B0C8X5GF7D,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$-0.00,$-0.00,$0.00,0.0%,100.0%,9,11.11%
637,B0C8X67WC1,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$-0.00,$-0.00,$0.00,0.0%,100.0%,38,2.63%
638,B0CLYX3YVS,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$-0.00,$-0.00,$0.00,0.0%,67.72%,162,0.62%
639,B0DJPS1J7B,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.94,$-0.00,$-0.00,$-0.94,0.0%,100.0%,582,0.17%


In [ ]:
asin_info_df = pd.read_excel('/content/Brands and ASINs list.xlsx')

# Step 1: Extract the first 5 words from the Description column
asin_info_df['Description (5 words only)'] = (
    asin_info_df['Description']
    .astype(str)  # Ensure text format
    .apply(lambda x: ' '.join(x.strip().split()[:5]))
)


# Step 2: Strip whitespace from all column names (good practice)
final_df_merged_with_total.columns = final_df_merged_with_total.columns.str.strip()
asin_info_df.columns = asin_info_df.columns.str.strip()

# Step 3: Ensure both DataFrames have a column named exactly 'ASIN'
# Rename from any similar variants (e.g., 'Asin', 'asin', etc.)
for df in [final_df_merged_with_total, asin_info_df]:
    asin_col = [col for col in df.columns if col.strip().lower() == 'asin']
    if asin_col and asin_col[0] != 'ASIN':
        df.rename(columns={asin_col[0]: 'ASIN'}, inplace=True)

# Step 4: Merge final_df_merged_with_total INTO asin_info_df
# We use a LEFT JOIN to keep all rows from asin_info_df and enrich with metrics
merged_result = final_df_merged_with_total.merge(
    asin_info_df,
    on='ASIN',
    how='left'
)

# Step 5: Reorder columns
final_columns_order = [
    'GM',
    'Brands',
    'ASIN',
    'Description (5 words only)',
    'quantity',
    'Amazon Top-line Sales (ATS)',
    'Product Cost',
    'Referral Fee',
    'FBA Fulfillment Fee',
    'other transaction fees',
    'Shipping/Kitting Fees',
    'Labeling/Polybagging Fees',
    'Storage Fees',
    'Allocated fees (Premium Services Fee + Subscription)',
    'Cost of Returns',
    'Gross Profit',
    'Gross Margin',
    'Featured Offer (Buy Box) Percentage',
    'Sessions - Total',
    'Unit Session Percentage'
]

# Step 6: Filter to only those columns that exist to avoid KeyError
available_columns = [col for col in final_columns_order if col in merged_result.columns]
merged_result = merged_result[available_columns]
merged_result

,GM,Brands,ASIN,Description (5 words only),quantity,Amazon Top-line Sales (ATS),Product Cost,Referral Fee,FBA Fulfillment Fee,other transaction fees,Shipping/Kitting Fees,Labeling/Polybagging Fees,Storage Fees,Allocated fees (Premium Services Fee + Subscription),Cost of Returns,Gross Profit,Gross Margin,Featured Offer (Buy Box) Percentage,Sessions - Total,Unit Session Percentage
0,Daria,MWI-VIRBAC,B01E46PJCE,MOVOFLEX Joint Support Supplement for,"2,153","$136,654.77","$-90,942.72","$-19,274.58","$-9,299.90",$-0.00,$-0.00,$-0.00,$-40.16,$-580.96,$-134.14,"$16,382.31",11.99%,92.84%,"59,330",61.97%
1,Daria,MWI-VIRBAC,B01E46WJIG,MOVOFLEX Joint Support Supplement for,"1,595","$87,232.27","$-58,042.05","$-12,359.24","$-6,484.74",$-0.00,$-0.00,$-0.00,$-14.26,$-370.85,$-167.79,"$9,793.34",11.23%,94.2%,"55,537",51.22%
2,Daria,MWI-VIRBAC,B01E44ZX9U,MOVOFLEX Joint Support Supplement for,"1,293","$93,948.52","$-62,568.27","$-13,218.54","$-6,090.84",$-0.00,$-0.00,$-0.00,$-33.76,$-399.40,$-79.66,"$11,558.05",12.3%,92.75%,"43,542",55.7%
3,Candace,WRAP BUDDIES,B08MFNJSPG,Wrap Buddies Wrapping Paper Clamps,27,$667.23,$-343.17,$-100.11,$-112.05,$-0.00,$-0.00,$-0.00,$-9.34,$-2.84,$-188.19,$-88.47,-13.26%,96.35%,"195,716",10.03%
4,Daria,MWI-VIRBAC,B0BHC99KGY,MOVOFLEX Advanced Joint Support Supplement,"1,745","$112,884.05","$-76,291.40","$-16,132.85","$-7,549.79",$-0.00,$-0.00,$-0.00,$-26.02,$-479.90,$-176.55,"$12,227.54",10.83%,97.37%,"54,908",33.75%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,Rich,GALWAY,B0C8X5GF7D,All-Weather Men's Golf Pants -,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$-0.00,$-0.00,$0.00,0.0%,100.0%,9,11.11%
637,Rich,GALWAY,B0C8X67WC1,All-Weather Men's Golf Pants -,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$-0.00,$-0.00,$0.00,0.0%,100.0%,38,2.63%
638,Jocelyn,SWIMLINE,B0CLYX3YVS,SOLSTICE Scout 1 to 2,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$-0.00,$-0.00,$0.00,0.0%,67.72%,162,0.62%
639,Daria,NADRI,B0DJPS1J7B,NADRI Brooches and Pins For,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.94,$-0.00,$-0.00,$-0.94,0.0%,100.0%,582,0.17%


In [ ]:
asin_brand_df

,Brands,ASIN,Description,GM,Shipping %,Returns %
0,VIROX,B00B9G3ZJM,REScue One-Step Disinfectant Cleaner & Deodori...,Jocelyn,1,1.0
1,MWI-VIRBAC,B01E46PJCE,MOVOFLEX Joint Support Supplement for Dogs - H...,Daria,0,1.0
2,MWI-VIRBAC,B0BHC99KGY,MOVOFLEX Advanced Joint Support Supplement for...,Daria,0,1.0
3,WABASH VALLEY FARMS,B07Y3X733T,Original Whirley Pop Popcorn Maker - Wabash Va...,Candace,1,0.5
4,MWI-VIRBAC,B0BHBW54WV,MOVOFLEX Advanced Joint Support Supplement for...,Daria,0,1.0
...,...,...,...,...,...,...
2307,WHITE KNIGHT,B0BGVF49S4,White Knight WN13307-20AM Chrome M12 x 1.50 Fa...,Rich,1,1.0
2308,WHITE KNIGHT,B0BGVG7KB1,NaN,Rich,1,1.0
2309,WHITE KNIGHT,B0BGVHFY7V,NaN,Rich,1,1.0
2310,WHITE KNIGHT,B0BGVNTKTL,NaN,Rich,1,1.0


In [ ]:
asin_brand_df.columns = asin_brand_df.columns.str.strip()  # Trim whitespace
asin_brand_df.rename(columns=lambda x: x.strip(), inplace=True)  # Extra safety


In [ ]:
asin_brand_df['ASIN']

,ASIN
0,B00B9G3ZJM
1,B01E46PJCE
2,B0BHC99KGY
3,B07Y3X733T
4,B0BHBW54WV
...,...
2307,B0BGVF49S4
2308,B0BGVG7KB1
2309,B0BGVHFY7V
2310,B0BGVNTKTL


In [ ]:
# Step 9: Clean dollar signs and convert to numeric
merged_result['Shipping/Kitting Fees'] = (
    merged_result['Shipping/Kitting Fees']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)

merged_result['Cost of Returns'] = (
    merged_result['Cost of Returns']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)


In [ ]:
# Step 7: Clean ASIN columns and percentage fields
asin_brand_df.columns = asin_brand_df.columns.str.strip()
merged_result.columns = merged_result.columns.str.strip()

asin_brand_df['ASIN'] = asin_brand_df['ASIN'].astype(str).str.strip().str.upper()
merged_result['ASIN'] = merged_result['ASIN'].astype(str).str.strip().str.upper()


# Step 8: Merge on ASIN
merged_result = merged_result.merge(
    asin_brand_df[['ASIN', 'Shipping %', 'Returns %']],
    on='ASIN',
    how='left'
)

# Step 9: Clean and convert original fee columns to numeric
merged_result['Shipping/Kitting Fees'] = (
    merged_result['Shipping/Kitting Fees']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)

merged_result['Cost of Returns'] = (
    merged_result['Cost of Returns']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)

# Step 10: Calculate adjusted fees
merged_result['Adjusted Shipping/Kitting Fees'] = merged_result['Shipping/Kitting Fees'] * merged_result['Shipping %']
merged_result['Adjusted Cost of Returns'] = merged_result['Cost of Returns'] * merged_result['Returns %']

# Step 11: Reorder adjusted columns
def insert_column_after(df, after_column, new_column_name):
    cols = df.columns.tolist()
    if after_column in cols and new_column_name in df.columns:
        idx = cols.index(after_column)
        cols.insert(idx + 1, cols.pop(cols.index(new_column_name)))
        df = df[cols]
    return df

merged_result = insert_column_after(merged_result, 'Shipping/Kitting Fees', 'Adjusted Shipping/Kitting Fees')
merged_result = insert_column_after(merged_result, 'Cost of Returns', 'Adjusted Cost of Returns')

# Step 12: Calculate totals for adjusted columns
adjusted_shipping_total = merged_result.loc[merged_result['ASIN'] != 'TOTAL', 'Adjusted Shipping/Kitting Fees'].sum()
adjusted_returns_total = merged_result.loc[merged_result['ASIN'] != 'TOTAL', 'Adjusted Cost of Returns'].sum()

# Step 13: Format all fee columns with $ and 2 decimal places
merged_result['Shipping/Kitting Fees'] = merged_result['Shipping/Kitting Fees'].apply(
    lambda x: f"${x:,.2f}" if pd.notna(x) else ""
)

merged_result['Cost of Returns'] = merged_result['Cost of Returns'].apply(
    lambda x: f"${x:,.2f}" if pd.notna(x) else ""
)

merged_result['Adjusted Shipping/Kitting Fees'] = merged_result['Adjusted Shipping/Kitting Fees'].apply(
    lambda x: f"${x:,.2f}" if pd.notna(x) else ""
)

merged_result['Adjusted Cost of Returns'] = merged_result['Adjusted Cost of Returns'].apply(
    lambda x: f"${x:,.2f}" if pd.notna(x) else ""
)

# Step 14: Update the existing total row
total_index = merged_result[merged_result['ASIN'] == 'TOTAL'].index
if not total_index.empty:
    merged_result.loc[total_index, 'Adjusted Shipping/Kitting Fees'] = f"${adjusted_shipping_total:,.2f}"
    merged_result.loc[total_index, 'Adjusted Cost of Returns'] = f"${adjusted_returns_total:,.2f}"

# Step 15: Drop temp columns
merged_result.drop(columns=['Shipping %', 'Returns %'], inplace=True)


In [ ]:
merged_result

,GM,Brands,ASIN,Description (5 words only),quantity,Amazon Top-line Sales (ATS),Product Cost,Referral Fee,FBA Fulfillment Fee,other transaction fees,...,Labeling/Polybagging Fees,Storage Fees,Allocated fees (Premium Services Fee + Subscription),Cost of Returns,Adjusted Cost of Returns,Gross Profit,Gross Margin,Featured Offer (Buy Box) Percentage,Sessions - Total,Unit Session Percentage
0,Daria,MWI-VIRBAC,B01E46PJCE,MOVOFLEX Joint Support Supplement for,"2,153","$136,654.77","$-90,942.72","$-19,274.58","$-9,299.90",$-0.00,...,$-0.00,$-40.16,$-580.96,$-134.14,$-134.14,"$16,382.31",11.99%,92.84%,"59,330",61.97%
1,Daria,MWI-VIRBAC,B01E46WJIG,MOVOFLEX Joint Support Supplement for,"1,595","$87,232.27","$-58,042.05","$-12,359.24","$-6,484.74",$-0.00,...,$-0.00,$-14.26,$-370.85,$-167.79,$-167.79,"$9,793.34",11.23%,94.2%,"55,537",51.22%
2,Daria,MWI-VIRBAC,B01E44ZX9U,MOVOFLEX Joint Support Supplement for,"1,293","$93,948.52","$-62,568.27","$-13,218.54","$-6,090.84",$-0.00,...,$-0.00,$-33.76,$-399.40,$-79.66,$-79.66,"$11,558.05",12.3%,92.75%,"43,542",55.7%
3,Candace,WRAP BUDDIES,B08MFNJSPG,Wrap Buddies Wrapping Paper Clamps,27,$667.23,$-343.17,$-100.11,$-112.05,$-0.00,...,$-0.00,$-9.34,$-2.84,$-188.19,$-47.05,$-88.47,-13.26%,96.35%,"195,716",10.03%
4,Daria,MWI-VIRBAC,B0BHC99KGY,MOVOFLEX Advanced Joint Support Supplement,"1,745","$112,884.05","$-76,291.40","$-16,132.85","$-7,549.79",$-0.00,...,$-0.00,$-26.02,$-479.90,$-176.55,$-176.55,"$12,227.54",10.83%,97.37%,"54,908",33.75%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,Rich,GALWAY,B0C8X5GF7D,All-Weather Men's Golf Pants -,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,...,$-0.00,$0.00,$-0.00,$-0.00,$-0.00,$0.00,0.0%,100.0%,9,11.11%
637,Rich,GALWAY,B0C8X67WC1,All-Weather Men's Golf Pants -,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,...,$-0.00,$0.00,$-0.00,$-0.00,$-0.00,$0.00,0.0%,100.0%,38,2.63%
638,Jocelyn,SWIMLINE,B0CLYX3YVS,SOLSTICE Scout 1 to 2,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,...,$-0.00,$0.00,$-0.00,$-0.00,$-0.00,$0.00,0.0%,67.72%,162,0.62%
639,Daria,NADRI,B0DJPS1J7B,NADRI Brooches and Pins For,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,...,$-0.00,$-0.94,$-0.00,$-0.00,$-0.00,$-0.94,0.0%,100.0%,582,0.17%


In [ ]:
print(merged_result.columns.tolist())

['GM', 'Brands', 'ASIN', 'Description (5 words only)', 'quantity', 'Amazon Top-line Sales (ATS)', 'Product Cost', 'Referral Fee', 'FBA Fulfillment Fee', 'other transaction fees', 'Shipping/Kitting Fees', 'Adjusted Shipping/Kitting Fees', 'Labeling/Polybagging Fees', 'Storage Fees', 'Allocated fees (Premium Services Fee + Subscription)', 'Cost of Returns', 'Adjusted Cost of Returns', 'Gross Profit', 'Gross Margin', 'Featured Offer (Buy Box) Percentage', 'Sessions - Total', 'Unit Session Percentage']


In [ ]:
import numpy as np

# Step 1: Clean adjusted columns and other financial fields to float
money_cols = [
    'Amazon Top-line Sales (ATS)', 'Product Cost', 'Referral Fee',
    'FBA Fulfillment Fee', 'other transaction fees',
    'Shipping/Kitting Fees', 'Adjusted Shipping/Kitting Fees',
    'Labeling/Polybagging Fees', 'Storage Fees',
    'Allocated fees (Premium Services Fee + Subscription)',
    'Cost of Returns', 'Adjusted Cost of Returns', 'Gross Profit'
]

for col in money_cols:
    if col in merged_result.columns:
        merged_result[col] = (
            merged_result[col]
            .astype(str)
            .str.replace('$', '', regex=False)
            .str.replace(',', '', regex=False)
            .replace('', np.nan)
            .astype(float)
        )

# Step 2: Recalculate Gross Profit using ADJUSTED values
merged_result['Gross Profit'] = (
    merged_result['Amazon Top-line Sales (ATS)']
    + merged_result['Product Cost']
    + merged_result['Referral Fee']
    + merged_result['FBA Fulfillment Fee']
    + merged_result['other transaction fees']
    + merged_result['Adjusted Shipping/Kitting Fees']
    + merged_result['Labeling/Polybagging Fees']
    + merged_result['Storage Fees']
    + merged_result['Allocated fees (Premium Services Fee + Subscription)']
    + merged_result['Adjusted Cost of Returns']
)

# Step 3: Recalculate Gross Margin (as a decimal)
merged_result['Gross Margin'] = np.where(
    merged_result['Amazon Top-line Sales (ATS)'] != 0,
    merged_result['Gross Profit']*100 / merged_result['Amazon Top-line Sales (ATS)'],
    0
)

# Step 4 (Optional): Round to 2 decimals (you can skip this if you want raw float)
for col in money_cols + ['Gross Margin']:
    if col in merged_result.columns:
        merged_result[col] = merged_result[col].round(2)

# Step 5: Format money columns as $ string
for col in money_cols:
    if col in merged_result.columns:
        merged_result[col] = merged_result[col].apply(
            lambda x: f"${x:,.2f}" if pd.notna(x) else ""
        )

# Step 6: Format Gross Margin as percentage string
merged_result['Gross Margin'] = merged_result['Gross Margin'].apply(
    lambda x: f"{x:.2f}%" if pd.notna(x) else ""
)


In [ ]:
merged_result

,GM,Brands,ASIN,Description (5 words only),quantity,Amazon Top-line Sales (ATS),Product Cost,Referral Fee,FBA Fulfillment Fee,other transaction fees,...,Labeling/Polybagging Fees,Storage Fees,Allocated fees (Premium Services Fee + Subscription),Cost of Returns,Adjusted Cost of Returns,Gross Profit,Gross Margin,Featured Offer (Buy Box) Percentage,Sessions - Total,Unit Session Percentage
0,Daria,MWI-VIRBAC,B01E46PJCE,MOVOFLEX Joint Support Supplement for,"2,153","$136,654.77","$-90,942.72","$-19,274.58","$-9,299.90",$-0.00,...,$-0.00,$-40.16,$-580.96,$-134.14,$-134.14,"$16,382.31",11.99%,92.84%,"59,330",61.97%
1,Daria,MWI-VIRBAC,B01E46WJIG,MOVOFLEX Joint Support Supplement for,"1,595","$87,232.27","$-58,042.05","$-12,359.24","$-6,484.74",$-0.00,...,$-0.00,$-14.26,$-370.85,$-167.79,$-167.79,"$9,793.34",11.23%,94.2%,"55,537",51.22%
2,Daria,MWI-VIRBAC,B01E44ZX9U,MOVOFLEX Joint Support Supplement for,"1,293","$93,948.52","$-62,568.27","$-13,218.54","$-6,090.84",$-0.00,...,$-0.00,$-33.76,$-399.40,$-79.66,$-79.66,"$11,558.05",12.30%,92.75%,"43,542",55.7%
3,Candace,WRAP BUDDIES,B08MFNJSPG,Wrap Buddies Wrapping Paper Clamps,27,$667.23,$-343.17,$-100.11,$-112.05,$-0.00,...,$-0.00,$-9.34,$-2.84,$-188.19,$-47.05,$52.67,7.89%,96.35%,"195,716",10.03%
4,Daria,MWI-VIRBAC,B0BHC99KGY,MOVOFLEX Advanced Joint Support Supplement,"1,745","$112,884.05","$-76,291.40","$-16,132.85","$-7,549.79",$-0.00,...,$-0.00,$-26.02,$-479.90,$-176.55,$-176.55,"$12,227.54",10.83%,97.37%,"54,908",33.75%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,Rich,GALWAY,B0C8X5GF7D,All-Weather Men's Golf Pants -,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,...,$-0.00,$0.00,$-0.00,$-0.00,$-0.00,$0.00,0.00%,100.0%,9,11.11%
637,Rich,GALWAY,B0C8X67WC1,All-Weather Men's Golf Pants -,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,...,$-0.00,$0.00,$-0.00,$-0.00,$-0.00,$0.00,0.00%,100.0%,38,2.63%
638,Jocelyn,SWIMLINE,B0CLYX3YVS,SOLSTICE Scout 1 to 2,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,...,$-0.00,$0.00,$-0.00,$-0.00,$-0.00,$0.00,0.00%,67.72%,162,0.62%
639,Daria,NADRI,B0DJPS1J7B,NADRI Brooches and Pins For,0,$0.00,$-0.00,$-0.00,$-0.00,$-0.00,...,$-0.00,$-0.94,$-0.00,$-0.00,$-0.00,$-0.94,0.00%,100.0%,582,0.17%


In [ ]:
asin_to_check = 'B07L9VHPVF'

asin_row = merged_result[merged_result['ASIN'] == asin_to_check]

# Show the relevant columns
asin_row


,GM,Brands,ASIN,Description (5 words only),quantity,Amazon Top-line Sales (ATS),Product Cost,Referral Fee,FBA Fulfillment Fee,other transaction fees,...,Labeling/Polybagging Fees,Storage Fees,Allocated fees (Premium Services Fee + Subscription),Cost of Returns,Adjusted Cost of Returns,Gross Profit,Gross Margin,Featured Offer (Buy Box) Percentage,Sessions - Total,Unit Session Percentage
23,Daria,MWI-VIRBAC,B07L9VHPVF,MOVOFLEX Joint Support Supplement for,218,"$30,091.42","$-20,077.80","$-4,216.26","$-1,292.42",$-0.00,...,$-0.00,$-22.24,$-127.93,$-9.86,$-9.86,"$4,344.91",14.44%,95.9%,"11,394",35.97%


use formular average for buybox and unit session for the total and add % for whole columns, put comma back + comma session total have to have comma and do the comma for quantity. product sales to gross profit in dollar format, add % to gross margin and 2 decimal points

In [ ]:
# Define the output file path
output_path = 'full_detailed_aggregated_report.xlsx'

# Export full_report to Excel
merged_result.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: full_detailed_aggregated_report.xlsx


In [ ]:
# Step 1: Make a numeric copy
df_numeric = merged_result.copy()

# Define columns
money_cols = [
    'Amazon Top-line Sales (ATS)', 'Product Cost', 'Referral Fee',
    'FBA Fulfillment Fee', 'other transaction fees',
    'Shipping/Kitting Fees', 'Adjusted Shipping/Kitting Fees',
    'Labeling/Polybagging Fees', 'Storage Fees',
    'Allocated fees (Premium Services Fee + Subscription)',
    'Cost of Returns', 'Adjusted Cost of Returns'
]  # Exclude Gross Profit for now

# Clean dollar values
for col in money_cols:
    df_numeric[col] = (
        df_numeric[col].astype(str)
        .str.replace('$', '', regex=False)
        .str.replace(',', '', regex=False)
        .replace('', np.nan)
        .astype(float)
    )

# Clean quantity
df_numeric['quantity'] = pd.to_numeric(df_numeric['quantity'], errors='coerce')

# Group and aggregate
agg_dict = {col: 'sum' for col in money_cols + ['quantity']}
merged_result_brand = df_numeric.groupby('Brands', as_index=False).agg(agg_dict)

# ✅ Step 2: Recalculate Gross Profit after aggregation
merged_result_brand['Gross Profit'] = (
    merged_result_brand['Amazon Top-line Sales (ATS)']
    + merged_result_brand['Product Cost']
    + merged_result_brand['Referral Fee']
    + merged_result_brand['FBA Fulfillment Fee']
    + merged_result_brand['other transaction fees']
    + merged_result_brand['Adjusted Shipping/Kitting Fees']
    + merged_result_brand['Labeling/Polybagging Fees']
    + merged_result_brand['Storage Fees']
    + merged_result_brand['Allocated fees (Premium Services Fee + Subscription)']
    + merged_result_brand['Adjusted Cost of Returns']
)

# ✅ Step 3: Recalculate Gross Margin (as a percentage)
merged_result_brand['Gross Margin'] = np.where(
    merged_result_brand['Amazon Top-line Sales (ATS)'] != 0,
    merged_result_brand['Gross Profit'] * 100 / merged_result_brand['Amazon Top-line Sales (ATS)'],
    0
)

# Step 4: Format money columns
for col in money_cols + ['Gross Profit']:
    merged_result_brand[col] = merged_result_brand[col].apply(
        lambda x: f"${x:,.2f}" if pd.notna(x) else ""
    )

# Format quantity
merged_result_brand['quantity'] = merged_result_brand['quantity'].apply(
    lambda x: f"{int(x):,}" if pd.notna(x) else ""
)

# Format Gross Margin as percentage
merged_result_brand['Gross Margin'] = merged_result_brand['Gross Margin'].apply(
    lambda x: f"{x:.2f}%" if pd.notna(x) else ""
)


In [ ]:
merged_result_brand

,Brands,Amazon Top-line Sales (ATS),Product Cost,Referral Fee,FBA Fulfillment Fee,other transaction fees,Shipping/Kitting Fees,Adjusted Shipping/Kitting Fees,Labeling/Polybagging Fees,Storage Fees,Allocated fees (Premium Services Fee + Subscription),Cost of Returns,Adjusted Cost of Returns,quantity,Gross Profit,Gross Margin
0,BIOVANTA,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$-0.77,$0.00,$0.00,$0.00,0,$-0.77,0.00%
1,BJORN,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,0,$0.00,0.00%
2,BUDDEEZ,$104.95,$-45.20,$-15.75,$-39.80,$0.00,$0.00,$0.00,$0.00,$-17.91,$-0.45,$-21.58,$-21.58,5,$-35.74,-34.05%
3,CANINE CAVIAR,$778.41,$-362.40,$-116.75,$-91.70,$0.00,$0.00,$0.00,$0.00,$-6.58,$-3.31,$-1.86,$-1.86,13,$195.81,25.16%
4,CUPIXEL,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,0,$0.00,0.00%
5,DENOVO,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$-0.83,$0.00,$0.00,$0.00,0,$-0.83,0.00%
6,GALWAY,$159.00,$-110.00,$-27.03,$-7.24,$0.00,$0.00,$0.00,$0.00,$-45.27,$-0.68,$-16.33,$-16.33,1,$-47.55,-29.91%
7,KIMBERBELL,"$3,795.98","$-1,997.87",$-529.95,$-296.38,$0.00,$-7.29,$0.00,$0.00,$-58.48,$-16.11,$-124.54,$-31.15,78,$866.04,22.81%
8,LEARNING DYNAMICS,"$24,360.90","$-11,123.00","$-3,832.50","$-1,847.93",$0.00,$-530.74,$0.00,$-122.83,$-101.76,$-103.56,"$-1,035.99",$-258.99,266,"$6,970.33",28.61%
9,MWI,"$3,286.68","$-1,738.66",$-492.44,$-667.83,$0.00,$0.00,$0.00,$-88.55,$-2.40,$-13.97,$-16.58,$-16.58,162,$266.25,8.10%


In [ ]:
# Define the output file path
output_path = 'full_detailed_aggregated_report_brand.xlsx'

# Export full_report to Excel
merged_result_brand.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: full_detailed_aggregated_report_brand.xlsx


In [ ]:
# Step 1: Create numeric-safe copy
df_numeric_gm = merged_result.copy()

# Define monetary and key numeric columns
money_cols = [
    'Amazon Top-line Sales (ATS)', 'Product Cost', 'Referral Fee',
    'FBA Fulfillment Fee', 'other transaction fees',
    'Shipping/Kitting Fees', 'Adjusted Shipping/Kitting Fees',
    'Labeling/Polybagging Fees', 'Storage Fees',
    'Allocated fees (Premium Services Fee + Subscription)',
    'Cost of Returns', 'Adjusted Cost of Returns'
]

# Clean and convert money fields to float
for col in money_cols:
    df_numeric_gm[col] = (
        df_numeric_gm[col].astype(str)
        .str.replace('$', '', regex=False)
        .str.replace(',', '', regex=False)
        .replace('', np.nan)
        .astype(float)
    )

# Clean quantity
df_numeric_gm['quantity'] = pd.to_numeric(df_numeric_gm['quantity'], errors='coerce')

# Step 2: Aggregate by GM
agg_dict_gm = {col: 'sum' for col in money_cols + ['quantity']}
merged_result_gm = df_numeric_gm.groupby('GM', as_index=False).agg(agg_dict_gm)

# Step 3: Recalculate Gross Profit
merged_result_gm['Gross Profit'] = (
    merged_result_gm['Amazon Top-line Sales (ATS)']
    + merged_result_gm['Product Cost']
    + merged_result_gm['Referral Fee']
    + merged_result_gm['FBA Fulfillment Fee']
    + merged_result_gm['other transaction fees']
    + merged_result_gm['Adjusted Shipping/Kitting Fees']
    + merged_result_gm['Labeling/Polybagging Fees']
    + merged_result_gm['Storage Fees']
    + merged_result_gm['Allocated fees (Premium Services Fee + Subscription)']
    + merged_result_gm['Adjusted Cost of Returns']
)

# Step 4: Recalculate Gross Margin
merged_result_gm['Gross Margin'] = np.where(
    merged_result_gm['Amazon Top-line Sales (ATS)'] != 0,
    merged_result_gm['Gross Profit'] * 100 / merged_result_gm['Amazon Top-line Sales (ATS)'],
    0
)

# Step 5: Format currency fields
for col in money_cols + ['Gross Profit']:
    merged_result_gm[col] = merged_result_gm[col].apply(
        lambda x: f"${x:,.2f}" if pd.notna(x) else ""
    )

# Format quantity
merged_result_gm['quantity'] = merged_result_gm['quantity'].apply(
    lambda x: f"{int(x):,}" if pd.notna(x) else ""
)

# Format Gross Margin as percentage string
merged_result_gm['Gross Margin'] = merged_result_gm['Gross Margin'].apply(
    lambda x: f"{x:.2f}%" if pd.notna(x) else ""
)

In [ ]:
merged_result_gm

,GM,Amazon Top-line Sales (ATS),Product Cost,Referral Fee,FBA Fulfillment Fee,other transaction fees,Shipping/Kitting Fees,Adjusted Shipping/Kitting Fees,Labeling/Polybagging Fees,Storage Fees,Allocated fees (Premium Services Fee + Subscription),Cost of Returns,Adjusted Cost of Returns,quantity,Gross Profit,Gross Margin
0,Candace,"$3,024.38","$-1,483.84",$-453.34,$-605.11,$0.00,$0.00,$0.00,$0.00,"$-1,015.24",$-12.86,$-661.63,$-165.40,162,$-711.41,-23.52%
1,Daria,"$829,765.19","$-552,633.02","$-118,196.55","$-54,541.28",$0.00,$-672.97,$-97.57,$-577.20,$-389.69,"$-3,527.57","$-2,127.25","$-2,033.86","2,895","$97,768.45",11.78%
2,David M,"$22,428.09","$-10,406.30","$-3,363.42","$-5,850.84",$0.00,$0.00,$0.00,$-59.95,$-740.26,$-95.33,$-119.13,$-119.13,881,"$1,792.86",7.99%
3,David T,"$10,406.00","$-4,208.16","$-1,248.72",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$-44.24,$0.00,$0.00,114,"$4,904.88",47.14%
4,Jocelyn,"$310,530.50","$-145,107.92","$-45,923.56","$-41,868.72",$0.00,"$-1,216.74",$-686.00,$-460.18,"$-10,111.58","$-1,320.14","$-1,583.77",$-806.77,"6,120","$64,245.63",20.69%
5,Rich,"$4,329.04","$-2,256.26",$-651.97,$-806.57,$0.00,$0.00,$0.00,$-88.55,$-73.89,$-18.41,$-85.61,$-85.61,181,$347.78,8.03%


In [ ]:
# Define the output file path
output_path = 'full_detailed_aggregated_report_gm.xlsx'

# Export full_report to Excel
merged_result_gm.to_excel(output_path, index=False)

print(f"Export successful! File saved to: {output_path}")

Export successful! File saved to: full_detailed_aggregated_report_gm.xlsx


In [ ]:
# --- Configuration ---
column_mapping = {
    'Amazon Top-line Sales (ATS)': 'Amazon Top-line Sales (ATS)'
    # 'Product Cost': 'Current Wholesale Price',
    # 'Referral Fee': 'Referral Fee',
    # 'FBA Fulfillment Fee': 'TOTAL FBA Fulfillment Fee',
    # 'other transaction fees': 'other transaction fees',
    # 'Shipping/Kitting Fees': 'Shipping/Kitting Fees',
    # 'Labeling/Polybagging Fees': ['Amazon Labeling Fee', 'Amazon Bagging Fee'],
    # 'Storage Fees': 'Storage Fees',
    # 'Allocated fees (Premium Services Fee + Subscription)': 'Allocated fees (Premium Services Fee + Subscription)'
}
sales_cols = ['Amazon Top-line Sales (ATS)']


# --- Step 1: Clean and Normalize Data ---
merged_result['quantity'] = pd.to_numeric(merged_result['quantity'].replace(',', '', regex=True), errors='coerce')

for m_col, u_col in column_mapping.items():
    merged_result[m_col] = pd.to_numeric(merged_result[m_col].replace('[\$,]', '', regex=True), errors='coerce')

    if isinstance(u_col, list):
        for sub_col in u_col:
            unit_financial_df[sub_col] = pd.to_numeric(unit_financial_df[sub_col].replace('[\$,]', '', regex=True), errors='coerce')
        unit_financial_df[f'{m_col}_unit'] = unit_financial_df[u_col].sum(axis=1)
    else:
        unit_financial_df[u_col] = pd.to_numeric(unit_financial_df[u_col].replace('[\$,]', '', regex=True), errors='coerce')
        unit_financial_df[f'{m_col}_unit'] = unit_financial_df[u_col]

# --- Step 2: Compute Per-Unit Values and Compare ---
compare_df = merged_result[['ASIN']].copy()

for m_col in column_mapping:
    merged_result[f'{m_col}_per_unit'] = merged_result[m_col] / merged_result['quantity']
    compare_df[f'{m_col}_per_unit'] = merged_result[f'{m_col}_per_unit']
    compare_df = compare_df.merge(unit_financial_df[['ASIN', f'{m_col}_unit']], on='ASIN', how='left')

    compare_df[f'{m_col}_diff'] = compare_df[f'{m_col}_per_unit'] - compare_df[f'{m_col}_unit']

    if m_col in sales_cols:
        compare_df[f'{m_col}_flag'] = compare_df[f'{m_col}_diff'] < 0  # Sales too low
    else:
        # Actual fee (negative) is more than expected fee (positive), compare abs
        compare_df[f'{m_col}_flag'] = abs(compare_df[f'{m_col}_per_unit']) > abs(compare_df[f'{m_col}_unit'])

compare_df['any_issue'] = compare_df[[f'{col}_flag' for col in column_mapping]].any(axis=1)
problem_asins = compare_df[compare_df['any_issue']]['ASIN'].unique()

# --- Step 3: Enrich Orders with ASIN and Analyze ---
sku_asin_map = final_business_df[['SKU', '(Child) ASIN']].dropna()
clean_sku_asin_map = sku_asin_map.drop_duplicates(subset='SKU', keep='first')

orders_df = transaction_df[transaction_df['type'] == 'Order'].copy()
orders_df['product sales'] = pd.to_numeric(orders_df['product sales'].replace('[\$,]', '', regex=True), errors='coerce')
orders_df['quantity'] = pd.to_numeric(orders_df['quantity'], errors='coerce')

orders_with_asin = clean_sku_asin_map.merge(orders_df, how='right', left_on='SKU', right_on='sku')
orders_with_asin.rename(columns={'(Child) ASIN': 'ASIN'}, inplace=True)

orders_with_asin['unit_price'] = orders_with_asin['product sales'] / orders_with_asin['quantity']
unit_price_map = unit_financial_df.set_index('ASIN')['Amazon Top-line Sales (ATS)'].to_dict()
orders_with_asin['expected_unit_price'] = orders_with_asin['ASIN'].map(unit_price_map)
orders_with_asin['below_expected_price'] = orders_with_asin['unit_price'] < orders_with_asin['expected_unit_price']

In [ ]:
problem_asins

array(['B01E44ZX9U', 'B08MFNJSPG', 'B07W7646SJ', 'B004JR1YJQ',
       'B09NB1XG6T', 'B0BBP799GW', 'B09NCMGJCL', 'B09NF8QT94',
       'B0BBPYT5HH', 'B00C8RJ3WE', 'B0BBPS53L3', 'B0BBPVC7XK',
       'B00S46RU8W', 'B0CPHNSV71', 'B0BY9LDC26', 'B00C8RJ30Q',
       'B09NF9TM2H', 'B005JTY9RW', 'B0CPHNP1CQ', 'B09NF8L5Q4',
       'B0BBPJ4V7Q', 'B0CPHQM1VL', 'B00AT6FY6A', 'B00C8RJ3BA',
       'B0D6915M4B', 'B0827CX8CN', 'B084KV7N6J', 'B008DXYTGW',
       'B0DGB6DVZN', 'B0C9LLWYVK', 'B0BVN3GWLR', 'B0D692ZW9X',
       'B0DG9Y8RKB', 'B0CB9GD5DL', 'B0DGB5SFR4', 'B09DDBTYGV',
       'B00DCV38HQ', 'B00EACYOSI', 'B0BVN2C1XG', 'B0DG9X2BG3',
       'B09RJMD73B', 'B0BHXCC5F5', 'B0147Q84AK', 'B09DZ729W5',
       'B0BY9ZR6L5', 'B06XR4YPLZ', 'B09RJMYSL9', 'B09C6GCB6Y',
       'B0BSP6L4F7', 'B0DH6HB9K4', 'B0CYQYDVXD', 'B09RJN3DDC',
       'B0BHXBDHQH', 'B0CYQY5PGB', 'B0BBJCKH22', 'B01MTOFE7J',
       'B0BBJ8KYM6', 'B01MQJYZZP', 'B0CC7FZHJV', 'B084CZLWM5',
       'B08226P1N1', 'B08CVTHK9P', 'B0BB8H9N38'], dtype

In [ ]:
# --- Step 6: Investigate actual transaction details when Product Sales < Expected ATS ---
low_sales_transactions = []


# Setup for ATS
m_col = 'Amazon Top-line Sales (ATS)'
expected_col = f'{m_col}_unit'

# Map ASIN to expected per-unit ATS (from unit_financial_df)
ats_expected_map = unit_financial_df.set_index('ASIN')[expected_col].to_dict()

# Prepare transaction_df
txn_orders_df = transaction_df[transaction_df['type'] == 'Order'].copy()
txn_orders_df['product sales'] = pd.to_numeric(txn_orders_df['product sales'].replace('[\$,]', '', regex=True), errors='coerce')
txn_orders_df['quantity'] = pd.to_numeric(txn_orders_df['quantity'], errors='coerce')

# Merge SKU → ASIN
txn_orders_with_asin = clean_sku_asin_map.merge(txn_orders_df, how='right', left_on='SKU', right_on='sku')
txn_orders_with_asin.rename(columns={'(Child) ASIN': 'ASIN'}, inplace=True)
txn_orders_with_asin['average sales'] = txn_orders_with_asin['product sales'] / txn_orders_with_asin['quantity']

# Loop through ASINs flagged for low ATS
for asin in problem_asins:
    expected_per_unit = ats_expected_map.get(asin)
    if expected_per_unit is None:
        continue

    asin_txns = txn_orders_with_asin[txn_orders_with_asin['ASIN'] == asin].copy()
    if asin_txns.empty:
        continue

    asin_txns['expected_total_ats'] = expected_per_unit * asin_txns['quantity']

    # Show orders where actual product sales < expected ATS
    underperforming_sales = asin_txns[
        asin_txns['product sales'] < asin_txns['expected_total_ats']
    ]

    if not underperforming_sales.empty:
        low_sales_transactions.append(underperforming_sales)




# Combine all underperformers
if low_sales_transactions:
    ats_underperform_df = pd.concat(low_sales_transactions, ignore_index=True)

    # Check for presence of date/time column
    date_cols = [col for col in ats_underperform_df.columns if 'date/time' in col.lower()]
    display_columns = [
        'ASIN', 'sku', 'order id', 'quantity', 'product sales', 'average sales', 'expected_total_ats'
    ]
    if date_cols:
        display_columns.insert(3, date_cols[0])  # insert date column after 'order id'

    print("🛒 Underperforming product sales transactions:")
    display(ats_underperform_df[display_columns])
else:
    print("✅ No underperforming product sales transactions found (based on ATS expectation).")



🛒 Underperforming product sales transactions:


,ASIN,sku,order id,date/time,quantity,product sales,average sales,expected_total_ats
0,B01E44ZX9U,RC-VB-062096,113-6110630-4361047,"Feb 12, 2025 5:23:28 PM PST",1.0,0.00,0.00,72.69
1,B08MFNJSPG,RB-WRA-WBRD,111-6465853-7306650,"Feb 2, 2025 10:42:25 PM PST",1.0,23.74,23.74,24.99
2,B08MFNJSPG,RB-WRA-WBRD,112-2052860-7214604,"Feb 3, 2025 7:18:31 PM PST",1.0,23.74,23.74,24.99
3,B08MFNJSPG,RB-WRA-WBRD,112-2485326-6765801,"Feb 4, 2025 10:40:40 AM PST",1.0,23.74,23.74,24.99
4,B08MFNJSPG,RB-WRA-WBRD,112-6239599-8709831,"Feb 10, 2025 12:54:50 AM PST",1.0,23.74,23.74,24.99
...,...,...,...,...,...,...,...,...
3109,B08226P1N1,RC-V1-510072,111-2849657-9259465,"Feb 27, 2025 7:37:04 PM PST",2.0,29.98,14.99,33.98
3110,B08CVTHK9P,RB-LD-MathKit,114-5182150-5450601,"Feb 10, 2025 8:08:19 PM PST",1.0,48.98,48.98,49.00
3111,B0BB8H9N38,RB-SOL-44224,113-9297208-6275463,"Feb 19, 2025 3:51:43 PM PST",1.0,149.96,149.96,195.00
3112,B0BB8H9N38,RB-SOL-44224,113-9297208-6275463,"Feb 19, 2025 3:52:00 PM PST",1.0,149.96,149.96,195.00


In [ ]:
ats_underperform_df[display_columns]

,ASIN,sku,order id,date/time,quantity,product sales,average sales,expected_total_ats
0,B01E44ZX9U,RC-VB-062096,113-6110630-4361047,"Feb 12, 2025 5:23:28 PM PST",1.0,0.00,0.00,72.69
1,B08MFNJSPG,RB-WRA-WBRD,111-6465853-7306650,"Feb 2, 2025 10:42:25 PM PST",1.0,23.74,23.74,24.99
2,B08MFNJSPG,RB-WRA-WBRD,112-2052860-7214604,"Feb 3, 2025 7:18:31 PM PST",1.0,23.74,23.74,24.99
3,B08MFNJSPG,RB-WRA-WBRD,112-2485326-6765801,"Feb 4, 2025 10:40:40 AM PST",1.0,23.74,23.74,24.99
4,B08MFNJSPG,RB-WRA-WBRD,112-6239599-8709831,"Feb 10, 2025 12:54:50 AM PST",1.0,23.74,23.74,24.99
...,...,...,...,...,...,...,...,...
3109,B08226P1N1,RC-V1-510072,111-2849657-9259465,"Feb 27, 2025 7:37:04 PM PST",2.0,29.98,14.99,33.98
3110,B08CVTHK9P,RB-LD-MathKit,114-5182150-5450601,"Feb 10, 2025 8:08:19 PM PST",1.0,48.98,48.98,49.00
3111,B0BB8H9N38,RB-SOL-44224,113-9297208-6275463,"Feb 19, 2025 3:51:43 PM PST",1.0,149.96,149.96,195.00
3112,B0BB8H9N38,RB-SOL-44224,113-9297208-6275463,"Feb 19, 2025 3:52:00 PM PST",1.0,149.96,149.96,195.00


In [ ]:
# ✅ Replace with ASINs you want to inspect
check_asins = ['B074P6C27C']

# ✅ Check if ats_underperform_df exists
if 'ats_underperform_df' in locals():
    check_df = ats_underperform_df[ats_underperform_df['ASIN'].isin(check_asins)]

    if not check_df.empty:
        print(f"🔍 Underperforming sales details for ASINs: {check_asins}")
        display(check_df)
        print(check_df['quantity'].sum())
    else:
        print(f"⚠️ None of the specified ASINs found in underperforming transactions.")
else:
    print("❌ Variable 'ats_underperform_df' does not exist.")





⚠️ None of the specified ASINs found in underperforming transactions.


In [ ]:
# ✅ Export to Excel
output_path = "underperforming_ats_report_V2.xlsx"
ats_underperform_df[display_columns].to_excel(output_path, index=False)
print(f"✅ Report saved to: {output_path}")

✅ Report saved to: underperforming_ats_report_V2.xlsx


In [ ]:
!pip install XlsxWriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
# --- Step 1: Clean Gross Margin columns (drop “%”, coerce to float) ---
merged_result['Actual_GM'] = (
    merged_result['Gross Margin']
      .astype(str)
      .str.rstrip('%')
      .astype(float)
      .fillna(0.0)
)

unit_financial_df['Expected_GM'] = (
    unit_financial_df['Gross Margin']
      .astype(str)
      .str.rstrip('%')
      .astype(float)
).mul(100)

# --- Step 2: Build map of expected GM by ASIN ---
expected_gm_map = unit_financial_df.set_index('ASIN')['Expected_GM'].to_dict()

# --- Step 3: Join it back onto your merged_result ---
merged_result['Expected_GM'] = merged_result['ASIN'].map(expected_gm_map)

# --- Step 4: Filter to those with Actual < Expected ---
gm_below_df = merged_result.loc[
    merged_result['Actual_GM'] < merged_result['Expected_GM'],
    ['ASIN', 'Actual_GM', 'Expected_GM']
].copy()

# --- Optional: sort by difference, add a delta column ---
gm_below_df['Delta_GM'] = gm_below_df['Expected_GM'] - gm_below_df['Actual_GM']
gm_below_df = gm_below_df.sort_values('Delta_GM', ascending=False)

# --- Step 5: Rename for display and show ---
gm_below_df = gm_below_df.rename(columns={
    'Actual_GM':   'Actual Gross Margin (%)',
    'Expected_GM': 'Expected Gross Margin (%)',
    'Delta_GM':    'GM Gap (%)'
})
gm_below_df.index = range(1, len(gm_below_df)+1)
gm_below_df.index.name = 'No.'

print("⚠️ ASINs whose actual Gross Margin is below plan:")
display(gm_below_df)

# --- If you need to save out to Excel ---
output_file = "GM_Underperformers.xlsx"
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    gm_below_df.to_excel(writer, sheet_name='GM Below Plan')
print(f"✅ Exported to {output_file}")

⚠️ ASINs whose actual Gross Margin is below plan:


,ASIN,Actual Gross Margin (%),Expected Gross Margin (%),GM Gap (%)
No.,,,,
1,B00C8RJ3BA,-495.15,11.383923,506.533923
2,B0CF8248YC,-231.07,18.003755,249.073755
3,B0CF7R8CGS,-103.47,17.402402,120.872402
4,B0C9LLWYVK,-97.60,15.286951,112.886951
5,B008DXYTGW,-76.90,13.090738,89.990738
...,...,...,...,...
338,B0034MD6BQ,0.00,0.880383,0.880383
339,B07DTHKVQX,20.25,20.767029,0.517029
340,B0147Q84AK,19.43,19.815109,0.385109


✅ Exported to GM_Underperformers.xlsx


In [ ]:
# --- 1) Define fee columns to sum ---
fee_cols = [
    'Referral Fee',
    'FBA Fulfillment Fee',
    'other transaction fees',
    'Adjusted Shipping/Kitting Fees',
    'Labeling/Polybagging Fees',
    'Storage Fees',
    'Allocated fees (Premium Services Fee + Subscription)',
    'Adjusted Cost of Returns'
]

# --- 2) Ensure quantity is numeric (and keep it for “Units Sold”) ---
merged_result['quantity'] = (
    merged_result['quantity']
      .astype(str)
      .str.replace(",", "", regex=False)
)
merged_result['quantity'] = pd.to_numeric(
    merged_result['quantity'],
    errors='coerce'
).fillna(0.0)

# --- 3) Clean & convert each fee column to numeric safely ---
for col in fee_cols:
    if col in merged_result.columns:
        # strip $ and commas, replace empty strings with NaN, then coerce
        s = (merged_result[col]
               .astype(str)
               .str.replace(r"[\$,]", "", regex=True)
               .replace(r"^\s*$", np.nan, regex=True)
        )
        merged_result[col] = pd.to_numeric(s, errors='coerce').fillna(0.0)

# --- 4) Sum into actual_total_fees & compute per-unit fees ---
merged_result['Actual Total Fees'] = merged_result[fee_cols].sum(axis=1)
merged_result['Actual Fees per Unit'] = (
    merged_result['Actual Total Fees'] / merged_result['quantity']
).replace([np.inf, -np.inf], 0).fillna(0.0)

# --- 5) Negate unit_financial’s TOTAL AMAZON FEES and map expected per-unit fees ---
unit_financial_df['TOTAL AMAZON FEES'] = (
    unit_financial_df['TOTAL AMAZON FEES']
      .astype(str)
      .str.replace(r"[\$,]", "", regex=True)
)
unit_financial_df['TOTAL AMAZON FEES'] = pd.to_numeric(
    unit_financial_df['TOTAL AMAZON FEES'],
    errors='coerce'
).fillna(0.0) * -1.0

expected_map = unit_financial_df.set_index('ASIN')['TOTAL AMAZON FEES'].to_dict()
merged_result['Expected Fees per Unit'] = (
    merged_result['ASIN'].map(expected_map).fillna(0.0)
)

# --- 6) Filter ASINs where actual per-unit fees exceed (more negative) than expected ---
undercharged = merged_result.loc[
    merged_result['Actual Fees per Unit'] < merged_result['Expected Fees per Unit']
].copy()

# --- 7) Build the final table (including Units Sold) and drop the last row if needed ---
if not undercharged.empty:
    display_cols = (
        ['ASIN', 'quantity'] +
        fee_cols +
        ['Actual Total Fees', 'Actual Fees per Unit', 'Expected Fees per Unit']
    )
    result = undercharged[display_cols].rename(columns={
        'quantity':            'Units Sold',
    })

    # drop last row if >1 row
    if len(result) > 1:
        result = result.iloc[:-1]
    else:
        result = result.iloc[0:0]

    # show
    print("✅ ASINs where actual fees per unit exceed the unit-financial target:")
    display(result)

    # --- 8) Export for download ---
    output_file = "Fees_PerUnit_Underperformers.xlsx"
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        result.to_excel(writer, sheet_name='Fee_Per_Unit_Under', index=False)
    print(f"✅ Exported to {output_file}")

else:
    print("✅ No ASINs found where actual per-unit fees exceed expected TOTAL AMAZON FEES.")

✅ ASINs where actual fees per unit exceed the unit-financial target:


,ASIN,Units Sold,Referral Fee,FBA Fulfillment Fee,other transaction fees,Adjusted Shipping/Kitting Fees,Labeling/Polybagging Fees,Storage Fees,Allocated fees (Premium Services Fee + Subscription),Adjusted Cost of Returns,Actual Total Fees,Actual Fees per Unit,Expected Fees per Unit
0,B01E46PJCE,2153.0,-19274.58,-9299.90,-0.0,-0.0,-0.0,-40.16,-580.96,-134.14,-29329.74,-13.622731,15.319077
1,B01E46WJIG,1595.0,-12359.24,-6484.74,-0.0,-0.0,-0.0,-14.26,-370.85,-167.79,-19396.88,-12.161053,13.591940
2,B01E44ZX9U,1293.0,-13218.54,-6090.84,-0.0,-0.0,-0.0,-33.76,-399.40,-79.66,-19822.20,-15.330394,17.255588
3,B08MFNJSPG,27.0,-100.11,-112.05,-0.0,-0.0,-0.0,-9.34,-2.84,-47.05,-271.39,-10.051481,8.251170
4,B0BHC99KGY,1745.0,-16132.85,-7549.79,-0.0,-0.0,-0.0,-26.02,-479.90,-176.55,-24365.11,-13.962814,16.809079
...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,B0C7YS2VPL,0.0,-0.00,-0.00,-0.0,-0.0,-0.0,-10.06,-0.00,-0.00,-10.06,0.000000,3.007292
635,B0C8WWZQKR,0.0,-0.00,-0.00,-0.0,-0.0,-0.0,0.00,-0.00,-0.00,0.00,0.000000,9.281574
636,B0C8X5GF7D,0.0,-0.00,-0.00,-0.0,-0.0,-0.0,0.00,-0.00,-0.00,0.00,0.000000,9.281574
637,B0C8X67WC1,0.0,-0.00,-0.00,-0.0,-0.0,-0.0,0.00,-0.00,-0.00,0.00,0.000000,9.281574


✅ Exported to Fees_PerUnit_Underperformers.xlsx


In [ ]:
# # --- Step 5: Investigate actual transaction details when Selling Fee is higher than expected Referral Fee ---
# high_referral_transactions = []

# # Setup
# m_col = 'Referral Fee'
# expected_col = f'{m_col}_unit'

# # Map ASIN to expected per-unit referral fee
# referral_fee_expected_map = unit_financial_df.set_index('ASIN')[expected_col].to_dict()

# # Clean & prepare transaction_df
# txn_orders_df = transaction_df[transaction_df['type'] == 'Order'].copy()
# txn_orders_df['selling fees'] = pd.to_numeric(txn_orders_df['selling fees'].replace('[\$,]', '', regex=True), errors='coerce')
# txn_orders_df['quantity'] = pd.to_numeric(txn_orders_df['quantity'], errors='coerce')

# # Merge SKU → ASIN
# txn_orders_with_asin = clean_sku_asin_map.merge(txn_orders_df, how='right', left_on='SKU', right_on='sku')
# txn_orders_with_asin.rename(columns={'(Child) ASIN': 'ASIN'}, inplace=True)
# txn_orders_with_asin['average referral_fee'] = txn_orders_with_asin['Referral Fee'] / txn_orders_with_asin['quantity']

# # Loop through ASINs with referral fee issues
# for asin in problem_asins:
#     expected_per_unit = referral_fee_expected_map.get(asin)
#     if expected_per_unit is None:
#         continue

#     asin_txns = txn_orders_with_asin[txn_orders_with_asin['ASIN'] == asin].copy()
#     if asin_txns.empty:
#         continue

#     asin_txns['expected_total_referral_fee'] = expected_per_unit * asin_txns['quantity']

#     # Show transactions where actual selling fee > expected referral fee
#     overcharged_txns = asin_txns[
#         asin_txns['selling fees'] < -asin_txns['expected_total_referral_fee'] - epsilon
#     ]

#     if not overcharged_txns.empty:
#         high_referral_transactions.append(overcharged_txns)

# # Combine all
# if high_referral_transactions:
#     referral_fee_overcharged_df = pd.concat(high_referral_transactions, ignore_index=True)
#     display(referral_fee_overcharged_df[['ASIN', 'sku', 'order id', 'quantity', 'selling fees', 'expected_total_referral_fee']])
# else:
#     print("✅ No overcharged selling fee transactions found (compared to Referral Fee expectation).")


In [ ]:
print(unit_financial_df.columns.tolist())


['Brand Partner', 'UPC', 'SKU', 'ASIN', 'Product Name', 'Category', 'Trademark Brands', 'REVIEWED SIGN OFF', 'Fulfilled By', 'Unnamed: 9', 'OLD Wholesale Price', 'Current Wholesale Price', 'Amazon Labeling Fee', 'Amazon Bagging Fee', 'Shipping/Kitting Fees', 'Storage Fees', 'Cost of Returns', 'other transaction fees', 'Allocated fees (Premium Services Fee + Subscription)', 'Base FBA Fulfillment Fee', 'Amazon Weight Surcharge Multiple', 'Total Amazon Weight Surcharge', 'TOTAL FBA Fulfillment Fee', 'Referral Fee', 'TOTAL AMAZON FEES', 'Unnamed: 25', 'iMAP', 'MSRP', 'Amazon Top-line Sales (ATS)', 'Lead Referral Commission', 'Ad Co-Op', 'Gross Profit', 'Gross Margin', '# of Resellers', '2023 Units Sold', 'est sales', 'est profit', 'est margin', 'Max Weight', 'Amazon Shipping Weight', 'Unnamed: 40', 'Dim. Weight', 'Unnamed: 42', 'Classification', 'Unnamed: 44', 'Your Price', 'Sales Price', 'Longest Side', 'Median Side', 'Shortest Side', 'Length and Girth', 'Unnamed: 51', 'Item Package Weigh

In [ ]:
# --- Step 1: Clean referral fee and quantity ---
merged_result['Referral Fee'] = pd.to_numeric(merged_result['Referral Fee'], errors='coerce')
merged_result['quantity'] = pd.to_numeric(merged_result['quantity'], errors='coerce')
merged_result['quantity'] = merged_result['quantity'].replace(0, np.nan)

# --- Step 2: Compute actual per-unit referral fee (will be negative) ---
merged_result['actual_referral_fee_per_unit'] = merged_result['Referral Fee'] / merged_result['quantity']

# --- Step 3: Get expected FBA Referral Fee (positive → make negative) ---
unit_financial_df['Referral Fee'] = pd.to_numeric(unit_financial_df['Referral Fee'], errors='coerce').fillna(0)
expected_referral_map = (-1 * unit_financial_df.set_index('ASIN')['Referral Fee']).to_dict()


In [ ]:
unit_financial_df

,Brand Partner,UPC,SKU,ASIN,Product Name,Category,Trademark Brands,REVIEWED SIGN OFF,Fulfilled By,Unnamed: 9,...,Median Side,Shortest Side,Length and Girth,Unnamed: 51,Item Package Weight,Unnamed: 53,Actual Product Size Tier,Currency,Amazon Top-line Sales (ATS)_unit,Expected_GM
0,Animed,694244-97360-4,RB-ANI-097360,B07PH9Z1VG,AniFlex Complete HEMP - 16oz,Pet Products,Animed,NaN,FBA,NaN,...,4.00,4.00,16.00,inches,24.00,oz,Large Standard,USD,50.92,18.494850
1,Animed,694244-97361-1,RB-ANI-097361,B07QQKKVXQ,AniFlex Complete HEMP - 2.5 lb,Pet Products,Animed,NaN,FBA,NaN,...,5.00,5.00,20.00,inches,48.00,oz,Large Standard,USD,97.71,17.260584
2,Animed,694244-97362-8,RB-ANI-097362,0,AniFlex Complete HEMP - 5 lb,Pet Products,Animed,NaN,FBA,NaN,...,6.97,6.97,27.88,inches,88.00,oz,Large Standard,USD,186.96,17.015269
3,Animed,694244-97011-5,RB-ANI-097011,B07PYF9RZ6,Arthaway HEMP - 16oz,Pet Products,Animed,NaN,FBA,NaN,...,4.00,4.00,16.00,inches,24.00,oz,Large Standard,USD,46.71,17.162123
4,Animed,694244-97415-1,RB-ANI-097415,0,Glucosamine 5000 HEMP - 16oz,Pet Products,Animed,NaN,FBA,NaN,...,4.00,4.00,16.00,inches,24.00,oz,Large Standard,USD,25.49,10.993047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,CRICUT,2010171,TW-CRI-2010171,B0BTZZYF4P,EDIO SKINTONE SAMPLER 12X12 (10),Everything Else,CRICUT,NaN,FBA,NaN,...,1.81,1.81,7.24,inches,10.08,oz,Large Standard,USD,14.99,24.576772
1871,CRICUT,2010196,TW-CRI-2010196,B0BV11HL4J,EVDY IO CORAL 12X24 (1),Everything Else,CRICUT,NaN,FBA,NaN,...,1.75,1.75,7.00,inches,3.68,oz,Large Standard,USD,9.99,18.927010
1872,CRICUT,2010037,TW-CRI-2010037,B0BV18GVCK,LINERLESS TRANSFER TAPE 13INx30FT,Everything Else,CRICUT,NaN,FBA,NaN,...,2.00,2.00,8.00,inches,12.80,oz,Large Standard,USD,9.99,12.208486
1873,CRICUT,2008875,TW-CRI-2008875,B0CCJVLWR3,PERM MARKERS 2.5 BLACK (3),Everything Else,CRICUT,NaN,FBA,NaN,...,2.36,0.83,6.38,inches,2.56,oz,Large Standard,USD,9.99,21.468574


In [ ]:
# Check for a specific ASIN in final_merged
asin_to_check = 'B0BHBNLNM1'

result = unit_financial_df[unit_financial_df['ASIN'] == asin_to_check]

# Display the result
if not result.empty:
    print(result)
else:
    print(f"❌ ASIN {asin_to_check} not found in final_merged.")

     Brand Partner           UPC           SKU        ASIN  \
1029   MWI- Virbac  814514022324  RC-VB-118150  B0BHBNLNM1   

                 Product Name      Category Trademark Brands  \
1029  MovoFlex Advanced SM 60  Pet Products         Movoflex   

     REVIEWED SIGN OFF Fulfilled By  Unnamed: 9  ...  Median Side  \
1029               NaN          FBA         NaN  ...         2.75   

      Shortest Side  Length and Girth Unnamed: 51  Item Package Weight  \
1029           2.75              11.0      inches                  5.6   

      Unnamed: 53  Actual Product Size Tier  Currency  \
1029           oz            Large Standard       USD   

      Amazon Top-line Sales (ATS)_unit  Expected_GM  
1029                             55.73     5.731085  

[1 rows x 58 columns]


In [ ]:
merged_result['expected_referral_fee_per_unit'] = merged_result['ASIN'].map(expected_referral_map)

# --- Step 4: Filter ASINs where actual fee is more negative than expected ---
high_referral_fee_df = merged_result[
    (merged_result['expected_referral_fee_per_unit'].notna()) &
    (merged_result['actual_referral_fee_per_unit'] < merged_result['expected_referral_fee_per_unit'])
].copy()

# --- Step 5: Check and prepare transaction data ---
if not high_referral_fee_df.empty:
    print("⚠️ ASINs with higher (more negative) referral fee than expected:")
    display(high_referral_fee_df[['ASIN', 'quantity', 'Referral Fee',
                                  'actual_referral_fee_per_unit', 'expected_referral_fee_per_unit']])

    overcharged_asins = high_referral_fee_df['ASIN'].unique()

    # --- Step 6: Pull order transactions and attach ASIN ---
    order_transactions = transaction_df[transaction_df['type'] == 'Order'].copy()

    merged_orders = order_transactions.merge(
        unit_financial_df[['SKU', 'ASIN']],
        how='left',
        left_on='sku',
        right_on='SKU'
    )

    relevant_orders = merged_orders[merged_orders['ASIN'].isin(overcharged_asins)].copy()

    # Clean numeric fields
    for col in ['quantity', 'product sales', 'selling fee', 'fba fees']:
        if col in relevant_orders.columns:
            relevant_orders[col] = pd.to_numeric(relevant_orders[col], errors='coerce')

    # --- Step 7: Export both summary and transactions to Excel ---
    output_file = "Referral_Fee_Overcharge_Report.xlsx"
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        high_referral_fee_df[['ASIN', 'quantity', 'Referral Fee',
                              'actual_referral_fee_per_unit', 'expected_referral_fee_per_unit']] \
            .to_excel(writer, sheet_name='Referral Fee Summary', index=False)

        relevant_orders[['ASIN', 'date/time', 'order id', 'sku', 'description',
                         'quantity', 'product sales', 'selling fees', 'fba fees']] \
            .to_excel(writer, sheet_name='Order Transactions', index=False)

    print(f"✅ Exported referral overcharge details to: {output_file}")
else:
    print("✅ No ASINs found with higher referral fees than expected.")

⚠️ ASINs with higher (more negative) referral fee than expected:


,ASIN,quantity,Referral Fee,actual_referral_fee_per_unit,expected_referral_fee_per_unit
13,B074P6C27C,400.0,-1591.08,-3.977700,-3.9255
21,B0CF8248YC,32.0,-38.40,-1.200000,-1.1985
24,B0CF7R8CGS,26.0,-39.00,-1.500000,-1.4985
46,B07HWTT4WY,166.0,-2465.10,-14.850000,-12.5985
61,B0056EAKSM,131.0,-539.28,-4.116641,-3.9255
75,B012BSPHS8,37.0,-69.19,-1.870000,-1.7925
85,B0827CP8H8,16.0,-90.88,-5.680000,-5.6775
137,B0147QEC38,28.0,-315.00,-11.250000,-11.2485
149,B0CC7VJKZP,2.0,-7.50,-3.750000,-3.7470
152,B0147Q84AK,29.0,-326.25,-11.250000,-11.2485


✅ Exported referral overcharge details to: Referral_Fee_Overcharge_Report.xlsx


In [ ]:
# --- Check specific ASIN after all computations ---
asin_to_check = 'B0BHC99KGY'

# 1. Show comparison from merged_result
abc_row = merged_result[merged_result['ASIN'] == asin_to_check]
if not abc_row.empty:
    print(f"\n🔍 Referral Fee Comparison for ASIN '{asin_to_check}':")
    display(abc_row[['ASIN', 'quantity', 'Referral Fee',
                     'actual_referral_fee_per_unit', 'expected_referral_fee_per_unit']])
else:
    print(f"❌ ASIN '{asin_to_check}' not found in merged_result.")

# 2. Show if it's in the overcharged referral list
if 'high_referral_fee_df' in locals() and asin_to_check in high_referral_fee_df['ASIN'].values:
    print(f"\n⚠️ ASIN '{asin_to_check}' is in the overcharged referral fee list.")
else:
    print(f"✅ ASIN '{asin_to_check}' is NOT in the overcharged referral fee list.")

# 3. Show transaction-level info (assumes merged_orders already exists)
if 'merged_orders' in locals():
    relevant_orders = merged_orders[merged_orders['ASIN'] == asin_to_check]
    if not relevant_orders.empty:
        print(f"\n📦 Order-level breakdown for ASIN '{asin_to_check}':")
        display(relevant_orders[['ASIN', 'date/time', 'order id', 'sku', 'description',
                                 'quantity', 'product sales', 'selling fees', 'fba fees']])
    else:
        print(f"ℹ️ No order transactions found for ASIN '{asin_to_check}' in merged_orders.")
else:
    print("❌ 'merged_orders' not found — did you run the merge step earlier?")



🔍 Referral Fee Comparison for ASIN 'B0BHC99KGY':


,ASIN,quantity,Referral Fee,actual_referral_fee_per_unit,expected_referral_fee_per_unit
4,B0BHC99KGY,1745.0,-16132.85,-9.245186,-9.7035


✅ ASIN 'B0BHC99KGY' is NOT in the overcharged referral fee list.

📦 Order-level breakdown for ASIN 'B0BHC99KGY':


,ASIN,date/time,order id,sku,description,quantity,product sales,selling fees,fba fees
21,B0BHC99KGY,"Feb 1, 2025 12:13:29 AM PST",114-1935978-3872215,RC-VB-118163,MOVOFLEX Advanced Soft Chews Hip and Joint Sup...,1.0,64.69,-8.25,-4.15
23,B0BHC99KGY,"Feb 1, 2025 12:15:27 AM PST",111-1031747-0003401,RC-VB-118163,MOVOFLEX Advanced Soft Chews Hip and Joint Sup...,1.0,64.69,-9.70,-4.15
45,B0BHC99KGY,"Feb 1, 2025 12:45:34 AM PST",114-3224061-6149016,RC-VB-118163,MOVOFLEX Advanced Soft Chews Hip and Joint Sup...,1.0,64.69,-9.70,-4.15
77,B0BHC99KGY,"Feb 1, 2025 1:35:45 AM PST",113-5316976-5376258,RC-VB-118163,MOVOFLEX Advanced Soft Chews Hip and Joint Sup...,1.0,64.69,-9.70,-4.15
79,B0BHC99KGY,"Feb 1, 2025 1:39:53 AM PST",114-5050041-6013042,RC-VB-118163,MOVOFLEX Advanced Soft Chews Hip and Joint Sup...,1.0,64.69,-9.70,-4.15
...,...,...,...,...,...,...,...,...,...
20764,B0BHC99KGY,"Feb 28, 2025 10:59:20 PM PST",111-1037063-8762611,RC-VB-118163,MOVOFLEX Advanced Soft Chews Hip and Joint Sup...,1.0,64.69,-9.22,-4.51
20770,B0BHC99KGY,"Feb 28, 2025 11:06:45 PM PST",114-7494341-8721006,RC-VB-118163,MOVOFLEX Advanced Soft Chews Hip and Joint Sup...,1.0,64.69,-9.22,-4.51
20779,B0BHC99KGY,"Feb 28, 2025 11:14:33 PM PST",113-0882756-8332252,RC-VB-118163,MOVOFLEX Advanced Soft Chews Hip and Joint Sup...,1.0,64.69,-8.25,-4.51
20794,B0BHC99KGY,"Feb 28, 2025 11:31:20 PM PST",111-9472314-8125028,RC-VB-118163,MOVOFLEX Advanced Soft Chews Hip and Joint Sup...,1.0,64.69,-9.22,-4.51


In [ ]:
# --- Step 1: Clean fulfillment fee and quantity ---
merged_result['FBA Fulfillment Fee'] = pd.to_numeric(merged_result['FBA Fulfillment Fee'], errors='coerce')
merged_result['quantity'] = pd.to_numeric(merged_result['quantity'], errors='coerce')
merged_result['quantity'] = merged_result['quantity'].replace(0, np.nan)

# --- Step 2: Compute actual per-unit FBA Fulfillment Fee (will be negative) ---
merged_result['actual_fulfillment_fee_per_unit'] = merged_result['FBA Fulfillment Fee'] / merged_result['quantity']

# --- Step 3: Get expected per-unit fulfillment fee from unit_financial_df (make negative) ---
unit_financial_df['Base FBA Fulfillment Fee'] = pd.to_numeric(unit_financial_df['Base FBA Fulfillment Fee'], errors='coerce').fillna(0)
expected_fulfillment_map = (-1 * unit_financial_df.set_index('ASIN')['Base FBA Fulfillment Fee']).to_dict()
merged_result['expected_fulfillment_fee_per_unit'] = merged_result['ASIN'].map(expected_fulfillment_map)

# --- Step 4: Filter ASINs with higher (more negative) fulfillment fee than expected ---
high_fulfillment_fee_df = merged_result[
    (merged_result['expected_fulfillment_fee_per_unit'].notna()) &
    (merged_result['actual_fulfillment_fee_per_unit'] < merged_result['expected_fulfillment_fee_per_unit'])
].copy()

# --- Step 5: Check and prepare transaction data ---
if not high_fulfillment_fee_df.empty:
    print("⚠️ ASINs with higher (more negative) FBA Fulfillment Fee than expected:")
    display(high_fulfillment_fee_df[['ASIN', 'quantity', 'FBA Fulfillment Fee',
                                     'actual_fulfillment_fee_per_unit', 'expected_fulfillment_fee_per_unit']])

    overcharged_asins = high_fulfillment_fee_df['ASIN'].unique()

    # Step 6: Get all order transactions
    order_transactions = transaction_df[transaction_df['type'] == 'Order'].copy()

    # Step 6.1: Merge with unit_financial_df to get ASIN
    merged_orders_fulfill = order_transactions.merge(
        unit_financial_df[['SKU', 'ASIN']],
        how='left',
        left_on='sku',
        right_on='SKU'
    )

    # Step 6.2: Filter by overcharged ASINs
    relevant_orders = merged_orders_fulfill[merged_orders_fulfill['ASIN'].isin(overcharged_asins)].copy()

    # Step 6.3: Clean numeric fields
    for col in ['quantity', 'product sales', 'selling fees', 'fba fees']:
        if col in relevant_orders.columns:
            relevant_orders[col] = pd.to_numeric(relevant_orders[col], errors='coerce')

    # Step 7: Display transaction-level breakdown
    print("📦 Transaction-level breakdown for overcharged ASINs (FBA Fulfillment Fee):")
    display(relevant_orders[['ASIN', 'date/time', 'order id', 'sku', 'description',
                             'quantity', 'product sales', 'selling fees', 'fba fees']])

    # Optional: Save to Excel
    output_file = "FBA_Fulfillment_Fee_Overcharge_Report.xlsx"
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        high_fulfillment_fee_df[['ASIN', 'quantity', 'FBA Fulfillment Fee',
                                 'actual_fulfillment_fee_per_unit', 'expected_fulfillment_fee_per_unit']] \
            .to_excel(writer, sheet_name='Overcharge Summary', index=False)

        relevant_orders[['ASIN', 'date/time', 'order id', 'sku', 'description',
                         'quantity', 'product sales', 'selling fees', 'fba fees']] \
            .to_excel(writer, sheet_name='Order Transactions', index=False)

    print(f"✅ Exported FBA Fulfillment Fee overcharge details to: {output_file}")

else:
    print("✅ No ASINs found with higher FBA Fulfillment Fees than expected.")

⚠️ ASINs with higher (more negative) FBA Fulfillment Fee than expected:


,ASIN,quantity,FBA Fulfillment Fee,actual_fulfillment_fee_per_unit,expected_fulfillment_fee_per_unit
0,B01E46PJCE,2153.0,-9299.90,-4.319508,-4.15
1,B01E46WJIG,1595.0,-6484.74,-4.065668,-3.90
2,B01E44ZX9U,1293.0,-6090.84,-4.710626,-4.55
4,B0BHC99KGY,1745.0,-7549.79,-4.326527,-4.15
6,B0BHBW54WV,1195.0,-5634.10,-4.714728,-4.55
8,B07W7646SJ,77.0,-339.39,-4.407662,-4.15
9,B004JR1YJQ,485.0,-2385.53,-4.918619,-4.55
10,B09NB1XG6T,1078.0,-5897.92,-5.471169,-5.37
11,B07L9Q6ZYJ,411.0,-2271.15,-5.525912,-4.15
12,B07L9V5D7Z,350.0,-1646.43,-4.704086,-4.15


📦 Transaction-level breakdown for overcharged ASINs (FBA Fulfillment Fee):


,ASIN,date/time,order id,sku,description,quantity,product sales,selling fees,fba fees
1,B01E46WJIG,"Feb 1, 2025 12:02:30 AM PST",113-4797350-1501825,RC-VB-062094,MOVOFLEX Joint Support Supplement for Small Do...,1.0,54.69,-7.38,-3.90
2,B0D8579MXD,"Feb 1, 2025 12:02:34 AM PST",114-9871212-7250647,RB-VB-119329,Virbac ZENIDOG Gel Diffuser - Dog Calming Pher...,2.0,60.04,-9.00,-9.88
4,B07L9Q6ZYJ,"Feb 1, 2025 12:02:51 AM PST",113-8714462-7078632,RC-VB-062095-1-2pk,MoVoFlex Joint Support Supplement for Dogs - H...,1.0,120.59,-16.28,-5.37
6,B07W7646SJ,"Feb 1, 2025 12:04:37 AM PST",112-1297595-0411432,RB-WRA-WBGY,"Wrap Buddies Tabletop Gift Wrapping Tool, 2 Cl...",1.0,23.74,-3.56,-4.15
8,B07L9VHPVF,"Feb 1, 2025 12:06:06 AM PST",112-3988422-9121867,RC-VB-062096-2pk,MOVOFLEX Soft Chews Hip and Joint Support Supp...,1.0,138.11,-20.72,-5.77
...,...,...,...,...,...,...,...,...,...
20820,B0056EAKSM,"Feb 28, 2025 11:53:22 PM PST",111-8492894-3133837,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,-8.40,-4.15
20821,B074P6C27C,"Feb 28, 2025 11:53:22 PM PST",111-8492894-3133837,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,0.00,-4.15
20822,B0056EAKSM,"Feb 28, 2025 11:53:22 PM PST",111-8492894-3133837,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,0.00,-4.15
20823,B00S46RU8W,"Feb 28, 2025 11:53:26 PM PST",114-1332482-5954645,RC-V1-510059-1,Vet One DuoClenz Enzyme Coated Dog Dental Chew...,1.0,21.49,-3.22,-7.24


✅ Exported FBA Fulfillment Fee overcharge details to: FBA_Fulfillment_Fee_Overcharge_Report.xlsx


In [ ]:
asin_to_check = 'B074P6C27C'

# 1. Check in merged_result
abc_row = merged_result[merged_result['ASIN'] == asin_to_check]
if not abc_row.empty:
    print(f"\n🔍 FBA Fulfillment Fee Comparison for ASIN '{asin_to_check}':")
    display(abc_row[['ASIN', 'quantity', 'FBA Fulfillment Fee',
                     'actual_fulfillment_fee_per_unit', 'expected_fulfillment_fee_per_unit']])
else:
    print(f"❌ ASIN '{asin_to_check}' not found in merged_result.")

# 2. Check if it's flagged as overcharged
if asin_to_check in high_fulfillment_fee_df['ASIN'].values:
    print(f"\n⚠️ ASIN '{asin_to_check}' is overcharged on FBA Fulfillment Fee.")
else:
    print(f"✅ ASIN '{asin_to_check}' is NOT overcharged on FBA Fulfillment Fee.")

# 3. Show transaction-level data from merged_orders_fulfill
if 'merged_orders_fulfill' in locals():
    relevant_orders = merged_orders_fulfill[merged_orders_fulfill['ASIN'] == asin_to_check].copy()
    if not relevant_orders.empty:
        print(f"\n📦 Order-level breakdown for ASIN '{asin_to_check}':")
        display(relevant_orders[['ASIN', 'date/time', 'order id', 'sku', 'description',
                                 'quantity', 'product sales', 'selling fees', 'fba fees']])
    else:
        print(f"ℹ️ No order transactions found for ASIN '{asin_to_check}'.")
else:
    print("❌ 'merged_orders_fulfill' not defined. Please run the merge step first.")


🔍 FBA Fulfillment Fee Comparison for ASIN 'B074P6C27C':


,ASIN,quantity,FBA Fulfillment Fee,actual_fulfillment_fee_per_unit,expected_fulfillment_fee_per_unit
13,B074P6C27C,400.0,-1671.88,-4.1797,-4.15



⚠️ ASIN 'B074P6C27C' is overcharged on FBA Fulfillment Fee.

📦 Order-level breakdown for ASIN 'B074P6C27C':


,ASIN,date/time,order id,sku,description,quantity,product sales,selling fees,fba fees
128,B074P6C27C,"Feb 1, 2025 3:23:42 AM PST",111-0942672-6898651,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,-4.2,-4.15
416,B074P6C27C,"Feb 1, 2025 4:54:38 PM PST",112-6541077-2637021,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,-4.2,-4.15
461,B074P6C27C,"Feb 1, 2025 6:09:56 PM PST",113-3948623-4984260,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,20.69,-3.1,-4.15
552,B074P6C27C,"Feb 1, 2025 8:49:02 PM PST",112-5608930-0573030,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,-4.2,-4.15
569,B074P6C27C,"Feb 1, 2025 9:21:53 PM PST",114-0657984-2663427,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,-4.2,-4.15
...,...,...,...,...,...,...,...,...,...
20804,B074P6C27C,"Feb 28, 2025 11:37:39 PM PST",113-5113246-3742642,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,-8.4,-4.15
20806,B074P6C27C,"Feb 28, 2025 11:37:39 PM PST",113-5113246-3742642,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,0.0,-4.15
20817,B074P6C27C,"Feb 28, 2025 11:52:34 PM PST",112-3268366-9965846,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,-4.2,-4.15
20819,B074P6C27C,"Feb 28, 2025 11:53:22 PM PST",111-8492894-3133837,RC-VB-028254,Virbac Epi-Otic Advanced Ear Cleanser for Dogs...,1.0,27.99,-8.4,-4.15


In [ ]:
# --- Step 1: Clean Gross Margin ---
merged_result['Gross Margin'] = pd.to_numeric(
    merged_result['Gross Margin'], errors='coerce'
)

# --- Step 2: Build expected gross margin map (per unit) ---
unit_financial_df['Gross Margin'] = pd.to_numeric(
    unit_financial_df['Gross Margin'], errors='coerce'
).fillna(0)
expected_gm_map = unit_financial_df.set_index('ASIN')['Gross Margin'].to_dict()

# --- Step 3: Map expected GM onto the merged results ---
merged_result['expected_gm'] = merged_result['ASIN'].map(expected_gm_map)

# --- Step 4: Filter ASINs where actual GM is below expected GM ---
low_gm_df = merged_result[
    merged_result['expected_gm'].notna() &
    (merged_result['Gross Margin'] < merged_result['expected_gm'])
].copy()

if not low_gm_df.empty:
    print("⚠️ ASINs with gross margin below expected:")
    display(low_gm_df[[
        'ASIN',
        'Gross Margin',
        'expected_gm'
    ]])

    underperform_asins = low_gm_df['ASIN'].unique()

    # --- Step 5: Pull order transactions and attach ASIN ---
    order_transactions = transaction_df[transaction_df['type'] == 'Order'].copy()
    merged_orders = order_transactions.merge(
        unit_financial_df[['SKU', 'ASIN']],
        how='left',
        left_on='sku',
        right_on='SKU'
    )
    relevant_orders = merged_orders[
        merged_orders['ASIN'].isin(underperform_asins)
    ].copy()

    # Clean numeric columns
    for col in ['quantity', 'product sales', 'selling fees', 'fba fees']:
        if col in relevant_orders.columns:
            relevant_orders[col] = pd.to_numeric(
                relevant_orders[col], errors='coerce'
            )

    # --- Step 6: Export both summary and transactions to Excel ---
    output_file = "Gross_Margin_Underperform_Report.xlsx"
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        low_gm_df[[
            'ASIN',
            'Gross Margin',
            'expected_gm'
        ]].to_excel(writer, sheet_name='GM Summary', index=False)

        relevant_orders[[
            'ASIN',
            'date/time',
            'order id',
            'sku',
            'description',
            'quantity',
            'product sales',
            'selling fees',
            'fba fees'
        ]].to_excel(writer, sheet_name='Order Transactions', index=False)

    print(f"✅ Exported GM underperformance details to: {output_file}")
else:
    print("✅ No ASINs found with gross margin below expected.")


✅ No ASINs found with gross margin below expected.


In [ ]:
# --- Step 1: Clean "other transaction fees" and quantity ---
merged_result['other transaction fees'] = pd.to_numeric(merged_result['other transaction fees'], errors='coerce')
merged_result['quantity'] = pd.to_numeric(merged_result['quantity'], errors='coerce')
merged_result['quantity'] = merged_result['quantity'].replace(0, np.nan)

# --- Step 2: Compute actual per-unit other transaction fee (will be negative) ---
merged_result['actual_other_fee_per_unit'] = merged_result['other transaction fees'] / merged_result['quantity']

# --- Step 3: Get expected per-unit fee from unit_financial_df (make negative) ---
unit_financial_df['other transaction fees'] = pd.to_numeric(unit_financial_df['other transaction fees'], errors='coerce').fillna(0)
expected_other_fee_map = (-1 * unit_financial_df.set_index('ASIN')['other transaction fees']).to_dict()
merged_result['expected_other_fee_per_unit'] = merged_result['ASIN'].map(expected_other_fee_map)

# --- Step 4: Filter for overcharged ASINs ---
high_other_fee_df = merged_result[
    (merged_result['expected_other_fee_per_unit'].notna()) &
    (merged_result['actual_other_fee_per_unit'] < merged_result['expected_other_fee_per_unit'])
].copy()

# --- Step 5: Check and prepare transaction-level data ---
if not high_other_fee_df.empty:
    print("⚠️ ASINs with higher (more negative) 'other transaction fees' than expected:")
    display(high_other_fee_df[['ASIN', 'quantity', 'other transaction fees',
                               'actual_other_fee_per_unit', 'expected_other_fee_per_unit']])

    overcharged_asins = high_other_fee_df['ASIN'].unique()

    # Step 6: Filter 'Order' transactions
    order_transactions = transaction_df[transaction_df['type'] == 'Order'].copy()

    # Step 6.1: Merge with unit_financial_df to get ASIN
    merged_orders_other = order_transactions.merge(
        unit_financial_df[['SKU', 'ASIN']],
        how='left',
        left_on='sku',
        right_on='SKU'
    )

    # Step 6.2: Filter only matching ASINs
    relevant_orders = merged_orders_other[merged_orders_other['ASIN'].isin(overcharged_asins)].copy()

    # Step 6.3: Clean numeric fields
    for col in ['quantity', 'product sales', 'selling fees', 'fba fees', 'other transaction fees']:
        if col in relevant_orders.columns:
            relevant_orders[col] = pd.to_numeric(relevant_orders[col], errors='coerce')

    # Step 7: Display result
    print("📦 Order-level breakdown for overcharged ASINs (Other Transaction Fees):")
    display(relevant_orders[['ASIN', 'date/time', 'order id', 'sku', 'description',
                             'quantity', 'product sales', 'selling fees', 'fba fees', 'other transaction fees']])

else:
    print("✅ No ASINs found with higher 'other transaction fees' than expected.")


✅ No ASINs found with higher 'other transaction fees' than expected.


In [ ]:
# --- Step 1: Ensure numeric ---
merged_result['Storage Fees'] = pd.to_numeric(merged_result['Storage Fees'], errors='coerce')
merged_result['quantity'] = pd.to_numeric(merged_result['quantity'], errors='coerce')
merged_result['quantity'].replace(0, np.nan, inplace=True)

# --- Step 2: Actual per-unit storage fee ---
merged_result['storage_fee_per_unit'] = merged_result['Storage Fees'] / merged_result['quantity']

# --- Step 3: Expected per-unit storage fee (from unit_financial_df, make negative) ---
expected_map = unit_financial_df.set_index('ASIN')['Storage Fees'].to_dict()
merged_result['expected_storage_fee_per_unit'] = merged_result['ASIN'].map(expected_map) * -1

# --- Step 4: Find overcharged cases ---
overcharged = merged_result[
    (merged_result['expected_storage_fee_per_unit'].notna()) &
    (merged_result['storage_fee_per_unit'] < merged_result['expected_storage_fee_per_unit'])
].copy()

# --- Step 5: Display results ---
if not overcharged.empty:
    print("⚠️ Overcharged Storage Fees Detected:")
    display(overcharged[['ASIN', 'quantity', 'Storage Fees', 'storage_fee_per_unit', 'expected_storage_fee_per_unit']])

    for asin in overcharged['ASIN'].unique():
        print(f"\n📦 Long-Term Storage Transactions for ASIN: {asin}")
        lt_rows = lt_storage_df[lt_storage_df['asin'] == asin].copy()

        if not lt_rows.empty:
            lt_rows['expected-long term'] = 0
            lt_display = lt_rows[['asin', 'amount-charged', 'expected-long term']]

            total_row = pd.DataFrame([{
                'asin': asin,
                'amount-charged': lt_display['amount-charged'].sum(),
                'expected-long term': 0
            }])

            lt_display = pd.concat([lt_display, total_row], ignore_index=True)
            display(lt_display)
        else:
            print("No long-term storage fee data found.")

        print(f"\n📦 Short-Term Storage Summary for ASIN: {asin}")
        st_summary = storage_summary[storage_summary['asin'] == asin]
        if not st_summary.empty:
            display(st_summary[['asin', 'estimated_monthly_storage_fee', 'allocation %', 'fee $', 'average_quantity_on_hand']])
        else:
            print("No short-term storage summary found.")

        print(f"\n📦 Short-Term Storage Transactions for ASIN: {asin}")
        st_rows = storage_df[storage_df['asin'] == asin]
        if not st_rows.empty:
            display(st_rows[['asin', 'estimated_monthly_storage_fee', 'average_quantity_on_hand']])
        else:
            print("No short-term storage transaction data found.")

else:
    print("✅ No overcharged storage fees found.")


In [ ]:
# --- Optional: Check a specific ASIN manually after everything is computed ---
asin_to_check = 'B09WT2D35V'

if asin_to_check in overcharged['ASIN'].values:
    print(f"\n✅ ASIN '{asin_to_check}' is overcharged. Here's the breakdown:\n")

    abc_row = overcharged[overcharged['ASIN'] == asin_to_check]
    display(abc_row[['ASIN', 'quantity', 'Storage Fees', 'storage_fee_per_unit', 'expected_storage_fee_per_unit']])

    print(f"\n📦 Long-Term Storage Transactions for ASIN: {asin_to_check}")
    lt_rows = lt_storage_df[lt_storage_df['asin'] == asin_to_check].copy()
    if not lt_rows.empty:
        lt_rows['expected-long term'] = 0
        lt_display = lt_rows[['asin', 'amount-charged', 'expected-long term']]
        total_row = pd.DataFrame([{
            'asin': asin_to_check,
            'amount-charged': lt_display['amount-charged'].sum(),
            'expected-long term': 0
        }])
        lt_display = pd.concat([lt_display, total_row], ignore_index=True)
        display(lt_display)
    else:
        print("No long-term storage fee data found.")

    print(f"\n📦 Short-Term Storage Summary for ASIN: {asin_to_check}")
    st_summary = storage_summary[storage_summary['asin'] == asin_to_check]
    if not st_summary.empty:
        display(st_summary[['asin', 'estimated_monthly_storage_fee', 'allocation %', 'fee $', 'average_quantity_on_hand']])
    else:
        print("No short-term storage summary found.")

    print(f"\n📦 Short-Term Storage Transactions for ASIN: {asin_to_check}")
    st_rows = storage_df[storage_df['asin'] == asin_to_check]
    if not st_rows.empty:
        display(st_rows[['asin', 'estimated_monthly_storage_fee', 'average_quantity_on_hand']])
    else:
        print("No short-term storage transaction data found.")

else:
    print(f"ℹ️ ASIN '{asin_to_check}' is NOT in the overcharged list.")



✅ ASIN 'B09WT2D35V' is overcharged. Here's the breakdown:



,ASIN,quantity,Storage Fees,storage_fee_per_unit,expected_storage_fee_per_unit
481,B09WT2D35V,2.0,-11.34,-5.67,-0.030625



📦 Long-Term Storage Transactions for ASIN: B09WT2D35V


,asin,amount-charged,expected-long term
0,B09WT2D35V,10.64,0
1,B09WT2D35V,10.64,0



📦 Short-Term Storage Summary for ASIN: B09WT2D35V


,asin,estimated_monthly_storage_fee,allocation %,fee $,average_quantity_on_hand
130,B09WT2D35V,0.7195,0.01%,-0.7,72.96



📦 Short-Term Storage Transactions for ASIN: B09WT2D35V


,asin,estimated_monthly_storage_fee,average_quantity_on_hand
124,B09WT2D35V,0.0099,1.00
282,B09WT2D35V,0.0197,2.00
572,B09WT2D35V,0.0014,0.14
1146,B09WT2D35V,0.0099,1.00
1403,B09WT2D35V,0.0887,9.00
1619,B09WT2D35V,0.0394,4.00
1860,B09WT2D35V,0.0493,5.00
1884,B09WT2D35V,0.0278,2.82
2107,B09WT2D35V,0.2267,23.00
2279,B09WT2D35V,0.0099,1.00


In [ ]:
import numpy as np
import pandas as pd

# --- Step 1: Ensure numeric ---
merged_result['Storage Fees'] = pd.to_numeric(merged_result['Storage Fees'], errors='coerce')
merged_result['quantity'] = pd.to_numeric(merged_result['quantity'], errors='coerce')
merged_result['quantity'].replace(0, np.nan, inplace=True)

# --- Step 2: Actual per-unit storage fee ---
merged_result['storage_fee_per_unit'] = merged_result['Storage Fees'] / merged_result['quantity']

# --- Step 3: Expected per-unit storage fee (from unit_financial_df, make negative) ---
expected_map = unit_financial_df.set_index('ASIN')['Storage Fees'].to_dict()
merged_result['expected_storage_fee_per_unit'] = merged_result['ASIN'].map(expected_map) * -1

# --- Step 4: Find overcharged cases ---
overcharged = merged_result[
    (merged_result['expected_storage_fee_per_unit'].notna()) &
    (merged_result['storage_fee_per_unit'] < merged_result['expected_storage_fee_per_unit'])
].copy()

# --- Step 5: Export results ---
with pd.ExcelWriter("Overcharged_Storage_Report.xlsx", engine="xlsxwriter") as writer:
    if not overcharged.empty:
        # Summary sheet
        overcharged[['ASIN', 'quantity', 'Storage Fees', 'storage_fee_per_unit', 'expected_storage_fee_per_unit']] \
            .to_excel(writer, sheet_name="Overcharged Summary", index=False)

        for asin in overcharged['ASIN'].unique():
            # Long-Term Storage
            lt_rows = lt_storage_df[lt_storage_df['asin'] == asin].copy()
            if not lt_rows.empty:
                lt_rows['expected-long term'] = 0
                lt_display = lt_rows[['asin', 'amount-charged', 'expected-long term']]
                total_row = pd.DataFrame([{
                    'asin': asin,
                    'amount-charged': lt_display['amount-charged'].sum(),
                    'expected-long term': 0
                }])
                lt_display = pd.concat([lt_display, total_row], ignore_index=True)
                lt_display.to_excel(writer, sheet_name=f"{asin}_LT", index=False)

            # Short-Term Summary
            st_summary = storage_summary[storage_summary['asin'] == asin]
            if not st_summary.empty:
                st_summary[['asin', 'estimated_monthly_storage_fee', 'allocation %', 'fee $', 'average_quantity_on_hand']] \
                    .to_excel(writer, sheet_name=f"{asin}_ST_Summary", index=False)

            # Short-Term Storage Transactions
            st_rows = storage_df[storage_df['asin'] == asin]
            if not st_rows.empty:
                st_rows[['asin', 'estimated_monthly_storage_fee', 'average_quantity_on_hand']] \
                    .to_excel(writer, sheet_name=f"{asin}_ST_Details", index=False)
    else:
        print("✅ No overcharged storage fees found.")


<ipython-input-199-54e303d59de0>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_result['quantity'].replace(0, np.nan, inplace=True)


In [ ]:
# --- Step 1: Ensure numeric for comparison ---
merged_result['Shipping/Kitting Fees'] = pd.to_numeric(merged_result['Shipping/Kitting Fees'], errors='coerce')
merged_result['quantity'] = pd.to_numeric(merged_result['quantity'], errors='coerce')
merged_result['quantity'] = merged_result['quantity'].replace(0, np.nan)

# --- Step 2: Calculate actual per-unit Shipping/Kitting fee (will be negative)
merged_result['actual_shipping_fee_per_unit'] = merged_result['Shipping/Kitting Fees'] / merged_result['quantity']

# --- Step 3: Get expected per-unit fee and make it negative to match cost perspective
unit_financial_df['Shipping/Kitting Fees_unit'] = pd.to_numeric(unit_financial_df['Shipping/Kitting Fees_unit'], errors='coerce').fillna(0)
expected_shipping_map = (-1 * unit_financial_df.set_index('ASIN')['Shipping/Kitting Fees_unit']).to_dict()
merged_result['expected_shipping_fee_per_unit'] = merged_result['ASIN'].map(expected_shipping_map)

# --- Step 4: Detect where actual fee is more negative than expected ---
high_shipping_fee_df = merged_result[
    (merged_result['expected_shipping_fee_per_unit'].notna()) &
    (merged_result['actual_shipping_fee_per_unit'] < merged_result['expected_shipping_fee_per_unit'])
].copy()

# --- Step 5: Display result ---
if not high_shipping_fee_df.empty:
    print("⚠️ ASINs with higher (more negative) shipping/kitting fee than expected:")
    display(high_shipping_fee_df[['ASIN', 'quantity', 'Shipping/Kitting Fees',
                                   'actual_shipping_fee_per_unit', 'expected_shipping_fee_per_unit']])
else:
    print("✅ No ASINs found with higher shipping/kitting fees than expected.")

⚠️ ASINs with higher (more negative) shipping/kitting fee than expected:


,ASIN,quantity,Shipping/Kitting Fees,actual_shipping_fee_per_unit,expected_shipping_fee_per_unit
9,B004JR1YJQ,485.0,-446.20,-0.92,-0.0
27,B0CPHNSV71,210.0,-71.40,-0.34,-0.0
28,B0BY9LDC26,90.0,-56.70,-0.63,-0.0
34,B0CPHNP1CQ,165.0,-84.15,-0.51,-0.0
37,B0CPHQM1VL,145.0,-27.55,-0.19,-0.0
39,B07CTY1K6Q,106.0,-25.44,-0.24,-0.0
44,B0BY9MG2H7,13.0,-3.64,-0.28,-0.0
46,B07HWTT4WY,166.0,-258.96,-1.56,-0.0
50,B0BY9MWPX8,40.0,-24.00,-0.60,-0.0
59,B096L8QX8M,4.0,-0.92,-0.23,-0.0


In [ ]:
# ✅ Export to Excel
output_path = "high_shipping_fee.xlsx"
high_shipping_fee_df[['ASIN', 'quantity', 'Shipping/Kitting Fees',
                                   'actual_shipping_fee_per_unit', 'expected_shipping_fee_per_unit']].to_excel(output_path, index=False)
print(f"✅ Report saved to: {output_path}")

✅ Report saved to: high_shipping_fee.xlsx


In [ ]:
# --- Step 1: Ensure numeric ---
merged_result['Labeling/Polybagging Fees'] = pd.to_numeric(merged_result['Labeling/Polybagging Fees'], errors='coerce')
merged_result['quantity'] = pd.to_numeric(merged_result['quantity'], errors='coerce')
merged_result['quantity'] = merged_result['quantity'].replace(0, np.nan)

# --- Step 2: Compute actual per-unit labeling/polybagging fee (will be negative)
merged_result['actual_label_fee_per_unit'] = merged_result['Labeling/Polybagging Fees'] / merged_result['quantity']

# --- Step 3: Ensure expected fees are numeric and handle NaN ---
unit_financial_df['Amazon Labeling Fee'] = pd.to_numeric(unit_financial_df['Amazon Labeling Fee'], errors='coerce').fillna(0)
unit_financial_df['Amazon Bagging Fee'] = pd.to_numeric(unit_financial_df['Amazon Bagging Fee'], errors='coerce').fillna(0)

# --- Step 4: Compute expected per-unit labeling fee and make it negative ---
unit_financial_df['expected_label_fee_per_unit'] = -1 * (
    unit_financial_df['Amazon Labeling Fee'] + unit_financial_df['Amazon Bagging Fee']
)

# --- Step 5: Map expected fee to merged_result ---
expected_label_map = unit_financial_df.set_index('ASIN')['expected_label_fee_per_unit'].to_dict()
merged_result['expected_label_fee_per_unit'] = merged_result['ASIN'].map(expected_label_map)

# --- Step 6: Filter for overcharged label fees ---
high_label_fee_df = merged_result[
    (merged_result['expected_label_fee_per_unit'].notna()) &
    (merged_result['actual_label_fee_per_unit'] < merged_result['expected_label_fee_per_unit'])  # More negative = overcharged
].copy()

# --- Step 7: Display results ---
if not high_label_fee_df.empty:
    print("⚠️ ASINs with higher (more negative) labeling/polybagging fee than expected:")
    display(high_label_fee_df[['ASIN', 'quantity', 'Labeling/Polybagging Fees',
                               'actual_label_fee_per_unit', 'expected_label_fee_per_unit']])
else:
    print("✅ No ASINs found with higher labeling/polybagging fees than expected.")

⚠️ ASINs with higher (more negative) labeling/polybagging fee than expected:


,ASIN,quantity,Labeling/Polybagging Fees,actual_label_fee_per_unit,expected_label_fee_per_unit
9,B004JR1YJQ,485.0,-291.00,-0.60,-0.57
40,B00AT6FY6A,151.0,-188.75,-1.25,-0.57
62,B008DXYTGW,1.0,-0.59,-0.59,-0.57
74,B01MXXTX61,28.0,-16.80,-0.60,-0.57
147,B0BHXCC5F5,18.0,-10.80,-0.60,-0.57
198,B0BWL427XC,26.0,-15.60,-0.60,-0.57
229,B0BSP6L4F7,27.0,-15.93,-0.59,-0.57
269,B0147QF8O0,1.0,-0.60,-0.60,-0.57


In [ ]:
# ✅ Export to Excel
output_path = "high_label_fee.xlsx"
high_label_fee_df[['ASIN', 'quantity', 'Labeling/Polybagging Fees',
                               'actual_label_fee_per_unit', 'expected_label_fee_per_unit']].to_excel(output_path, index=False)
print(f"✅ Report saved to: {output_path}")

✅ Report saved to: high_label_fee.xlsx


In [ ]:
# --- Step 1: Ensure numeric ---
m_col = 'Allocated fees (Premium Services Fee + Subscription)'
expected_col = f'{m_col}_unit'

merged_result[m_col] = pd.to_numeric(merged_result[m_col], errors='coerce')
merged_result['quantity'] = pd.to_numeric(merged_result['quantity'], errors='coerce')
merged_result['quantity'] = merged_result['quantity'].replace(0, np.nan)

# --- Step 2: Calculate actual per-unit allocated fee (will be negative)
merged_result['actual_allocated_fee_per_unit'] = merged_result[m_col] / merged_result['quantity']

# --- Step 3: Load expected per-unit fee and make it negative for comparison
unit_financial_df[expected_col] = pd.to_numeric(unit_financial_df[expected_col], errors='coerce').fillna(0)
expected_allocated_map = (-1 * unit_financial_df.set_index('ASIN')[expected_col]).to_dict()
merged_result['expected_allocated_fee_per_unit'] = merged_result['ASIN'].map(expected_allocated_map)

# --- Step 4: Filter for overcharged ASINs ---
high_allocated_fee_df = merged_result[
    (merged_result['expected_allocated_fee_per_unit'].notna()) &
    (merged_result['actual_allocated_fee_per_unit'] < merged_result['expected_allocated_fee_per_unit'])
].copy()

# --- Step 5: Display results ---
if not high_allocated_fee_df.empty:
    print("⚠️ ASINs with higher (more negative) Allocated fees than expected:")
    display(high_allocated_fee_df[['ASIN', 'quantity', m_col,
                                   'actual_allocated_fee_per_unit', 'expected_allocated_fee_per_unit']])
else:
    print("✅ No ASINs found with higher allocated fees than expected.")

⚠️ ASINs with higher (more negative) Allocated fees than expected:


,ASIN,quantity,Allocated fees (Premium Services Fee + Subscription),actual_allocated_fee_per_unit,expected_allocated_fee_per_unit
3,B08MFNJSPG,27.0,-2.84,-0.105185,-0.00000
8,B07W7646SJ,77.0,-7.83,-0.101688,-0.00000
10,B09NB1XG6T,1078.0,-126.02,-0.116902,-0.00000
16,B0BBP799GW,940.0,-196.53,-0.209074,-0.00000
17,B09NCMGJCL,526.0,-58.42,-0.111065,-0.00000
18,B09NF8QT94,429.0,-47.21,-0.110047,-0.00000
19,B0BBPYT5HH,792.0,-164.51,-0.207715,-0.00000
21,B0CF8248YC,32.0,-1.09,-0.034063,-0.00000
22,B0BBPS53L3,703.0,-147.03,-0.209147,-0.00000
24,B0CF7R8CGS,26.0,-1.10,-0.042308,-0.00000


In [ ]:
# ✅ Export to Excel
output_path = "high_allocated_fee.xlsx"
high_allocated_fee_df[['ASIN', 'quantity', m_col,
                                   'actual_allocated_fee_per_unit', 'expected_allocated_fee_per_unit']].to_excel(output_path, index=False)
print(f"✅ Report saved to: {output_path}")

✅ Report saved to: high_allocated_fee.xlsx
